In [ ]:
import pandas as pd
import numpy as np
!pip install yfinance
import yfinance as yf
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from functools import reduce
from numpy.ma.core import append 
# import selenium # Por si hay que bajar las series de la BMV

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 1.3 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0


In [ ]:
%load_ext rpy2.ipython

!rm -rf RiesgosFinancieros
!git clone "https://github.com/BrandonRoberto/Tarea1-ARF"


Cloning into 'Tarea1-ARF'...
remote: Enumerating objects: 19, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 19 (delta 2), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (19/19), done.


In [ ]:
def CVaR(x , VaR):
  return np.mean(x[x >= VaR])

In [ ]:
def ExponentialWeighted(primer, num_esc):
  return [primer*((1-primer)**i) for i in range(num_esc)]

In [ ]:
def VaR_CA(x,P,alpha):
  aux = pd.DataFrame({"X":x, "Proba":P})
  aux = aux.sort_values(by = ["X"])
  aux["Acumulado"] = aux["Proba"].cumsum()
  aux2 = aux[aux["Acumulado"]>=(alpha)]
  return aux2["X"].values[0]

In [ ]:
def CVaR_CA(x,P,alpha):
  aux = pd.DataFrame({"X":x, "Proba":P})
  aux = aux.sort_values(by = ["X"])
  aux["Acumulado"] = aux["Proba"].cumsum()
  plist = aux["Acumulado"]/aux["Acumulado"].values[len(aux)-1]
  aux2 = aux[plist>alpha]
  loss = aux2["X"].values
  esc = aux2["Proba"].values
  return sum(loss*esc)/sum(esc)

1. Calcular el VaR y CVaR a un día por los métodos Simulación histórica con y sin alisado. Tomar como d ́ıa de valoración el 31 de agosto de 2022.

# Acciones


1000 acciones de GCarso, -5000 de America Móvil y 1200 WalMart. Todas de la BMV.

In [ ]:
class Riesgo_Equity_FX:

  def __init__(self):
    self.posiciones = None
    self.acciones = None
    self.VaR = None
    self.CVaR = None
    self.alpha = None
    self.summary = None
    self.alisado = None
  
  def compute(self,acciones, posiciones, fecha_valoracion, alpha = 0.98 , alisado = False):
    stock_data = yf.download(acciones, end = fecha_valoracion)
    X=stock_data.Close
    X = X.dropna()
    X = X[::-1]
    X0 = X.head(1)
    DeltaX = (X.shift(1)/X)-1
    DeltaX.iloc[0,:] = 0
    X_s = X0.values*(1+DeltaX)
    posiciones = pd.Series(posiciones,index = acciones)
    PL = (X_s-X0.values[0])*posiciones
    PL['Total']=PL.sum(axis=1)
    VaR_C=[]

    if alisado:
      c = 1
      PL.dropna(inplace = True)
      #PL = PL.iloc[::-1]
      PL["Esc_SA"] = ExponentialWeighted(0.05,PL.shape[0])
      col_aux = list(PL.columns)
      col_aux.remove("Esc_SA")
      vector_VaR = [VaR_CA(PL.iloc[:,i],PL.Esc_SA,alpha) for i in range(PL.shape[1]-1)]
      VaR = pd.Series(vector_VaR,index = col_aux)
    else:
      c = 0
      vector_VaR = [PL.iloc[:,i].quantile(1-alpha) for i in range(len(acciones))]
      VaR = pd.Series(vector_VaR,index = acciones)
      VaR = PL.quantile(alpha,numeric_only = True)

    
    for i in range(0,PL.shape[1]-c):
      VC=CVaR(PL.iloc[:,i],VaR[i])
      VaR_C.append(VC)

    self.summary = pd.DataFrame({"VaR":VaR})
    self.summary['CVaR'] = VaR_C

    self.posiciones = posiciones
    self.acciones = acciones
    self.VaR = self.summary[['VaR']]
    self.CVaR = self.summary[['CVaR']]
    self.alpha = alpha

    return


In [ ]:
misAcciones = Riesgo_Equity_FX()


## Sin Alisado

In [ ]:
misAcciones.compute(['GCARSOA1.MX','AMXL.MX','WALMEX.MX'], [1000, -5000, 1200], '2022-08-31', alpha = 0.98)

[*********************100%***********************]  3 of 3 completed


In [ ]:
fecha_valoracion = '2022-08-31'
acciones = ['GCARSOA1.MX','AMXL.MX','WALMEX.MX']
posiciones = [1000, -5000, 1200]
alpha = 0.98

stock_data = yf.download(acciones, end = fecha_valoracion)
X=stock_data.Close
X = X.dropna()
X = X[::-1]
X0 = X.head(1)
DeltaX = (X.shift(1)/X)-1
DeltaX.iloc[0,:] = 0
X_s = X0.values*(1+DeltaX)
posiciones = pd.Series(posiciones,index = acciones)
PL = (X_s-X0.values[0])*posiciones
PL['Total']=PL.sum(axis=1)
vector_VaR = [PL.iloc[:,i].quantile(1-alpha) for i in range(len(acciones))]
vector_VaR
VaR = pd.Series(vector_VaR,index = acciones)
VaR = PL.quantile(alpha,numeric_only = True)
VaR

[*********************100%***********************]  3 of 3 completed


AMXL.MX        2913.180519
GCARSOA1.MX    3679.314558
WALMEX.MX      2932.841036
Total          4611.037542
Name: 0.98, dtype: float64

In [ ]:
 X0*posiciones



,AMXL.MX,GCARSOA1.MX,WALMEX.MX
Date,,,
2022-08-30 00:00:00-05:00,-88549.995422,72699.996948,80892.004395


In [ ]:
misAcciones.summary

,VaR,CVaR
AMXL.MX,2913.180519,4324.671254
GCARSOA1.MX,3679.314558,5905.603465
WALMEX.MX,2932.841036,3953.015550
Total,4611.037542,6696.734639


## Con Alisado

In [ ]:
misAcciones.compute(['GCARSOA1.MX','AMXL.MX','WALMEX.MX'], [1000, -5000, 1200], '2022-08-31', alpha = 0.98, alisado=True)


[*********************100%***********************]  3 of 3 completed


In [ ]:
misAcciones.summary

,VaR,CVaR
AMXL.MX,2854.961201,4193.553839
GCARSOA1.MX,1855.731661,3237.213026
WALMEX.MX,2422.457910,3356.963723
Total,3359.957049,5013.779492


# Divisas

1500 dólares (peso dólar), 700 
euros, y -600 libras esterlinas.

In [ ]:
misDivisas = Riesgo_Equity_FX()

## Sin Alisado

In [ ]:
misDivisas.compute(['USDMXN=X', 'EURMXN=X', 'GBPMXN=X'], [1500, 700, -600], '2022-08-31')

[*********************100%***********************]  3 of 3 completed


In [ ]:
misDivisas.summary

,VaR,CVaR
EURMXN=X,303.964389,443.112141
GBPMXN=X,230.517204,305.599750
USDMXN=X,524.861190,808.684531
Total,559.169617,841.003245


In [ ]:
fecha_valoracion = '2022-08-31'
acciones = ['USDMXN=X', 'EURMXN=X', 'GBPMXN=X']
posiciones = [1500, 700, -600]
alpha = 0.98

stock_data = yf.download(acciones, end = fecha_valoracion)
X=stock_data.Close
X = X.dropna()
X = X[::-1]
X0 = X.head(1)
DeltaX = (X.shift(1)/X)-1
DeltaX.iloc[0,:] = 0
X_s = X0.values*(1+DeltaX)
posiciones = pd.Series(posiciones,index = acciones)
PL = (X_s-X0.values[0])*posiciones
PL['Total']=PL.sum(axis=1)
vector_VaR = [PL.iloc[:,i].quantile(1-alpha) for i in range(len(acciones))]
vector_VaR
VaR = pd.Series(vector_VaR,index = acciones)
VaR = PL.quantile(alpha,numeric_only = True)
VaR

[*********************100%***********************]  3 of 3 completed


EURMXN=X    303.964389
GBPMXN=X    230.517204
USDMXN=X    524.861190
Total       559.169617
Name: 0.98, dtype: float64

In [ ]:
posiciones*X0

,EURMXN=X,GBPMXN=X,USDMXN=X
Date,,,
2022-08-30 00:00:00+01:00,14010.84938,-14062.139511,29977.200508


## Con Alisado

In [ ]:
misDivisas.compute(['USDMXN=X', 'EURMXN=X', 'GBPMXN=X'], [1500, 700, -600], '2022-08-31', alisado = True)

[*********************100%***********************]  3 of 3 completed


In [ ]:
misDivisas.summary

,VaR,CVaR
EURMXN=X,138.388755,247.163040
GBPMXN=X,213.612424,289.685028
USDMXN=X,628.575052,949.987029
Total,603.457187,915.673160


Bonos

2 bonos largos, uno cup ́on cero de cetes (con la curva guber) con nocional de 1500 vencimiento de 180 d ́ıas, un
bono M con tasa fija de 6.5% con un cupón de 6.05% anual vencimiento de 3600 d ́ıas y nocional de 1000, y un
bono corto 1000 bondes con plazo 707 con cup ́on que paga 28 d ́ıas (calcular cup ́on basado en curva de fondeo y
 descontar con la curva correspondiente).

# CETE

In [ ]:
def Cete(tasa, t): # días del bono t = (fecha_entrega - fecha_valoración).days
  return 10/(1 + tasa*(t/360))

In [ ]:
data_guber = "Tarea1-ARF/tasa_guber_st.txt"
df= pd.read_table(data_guber)
df
tasa_cete=df.iloc[0][5]

In [ ]:
cete= Cete(tasa_cete,180)
Vcete=cete*1500
Vcete

14801.655499843018

In [ ]:
class Riesgo_Cete:

  def __init__(self):
    self.posicion = None
    self.fecha_valoracion = None
    self.fecha_entrega = None
    self.plazo = None
    self.VaR = None
    self.CVaR = None
    self.alpha = None
    # self.summary = None
    
  
  def compute(self,curva_tasa, dias, posicion, alpha):
    """
    - curva_tasa: DataFrame
    """
    # CETE
    self.plazo = dias # REVISAR
    ult_tasa = curva_tasa[str(self.plazo)][0]
    df = pd.DataFrame({'Tasa':curva_tasa[str(self.plazo)]})
    df['Ret_Tasa'] = (df['Tasa'].shift(1)/df['Tasa']) - 1
    df['Ret_Tasa'][0] = 0
    print(ult_tasa)
    print(df)
    df['Tasa_Act'] = ult_tasa*(1+df['Ret_Tasa'])
    df['CETE'] = posicion*df['Tasa_Act'].apply(Cete, args = dias)
    ult_cete = df['CETE'][0]
    df['CETE_PL'] = ult_cete - df['CETE']
    self.VaR = df['CETE_PL'].quantile(alpha, numeric_only=False)
    self.CVaR = CVaR(df['CETE_PL'],self.VaR)
    self.alpha = alpha
    self.posicion = posicion
    self.fecha_valoracion = fval
    self.fecha_entrega = fent

In [ ]:
miCETE = Riesgo_Cete()

In [ ]:
btasadesc="Tarea1-ARF/tasa_yield.txt"
df= pd.read_table(btasadesc)
df

,DATE,1,7,30,90,180,270,360,720,1080,...,1800,2160,2520,2880,3240,3600,5400,7200,9000,10800
0,20220831,0.084890,0.083771,0.082475,0.083958,0.084098,0.083840,0.083257,0.080757,0.081060,...,0.080764,0.080245,0.079846,0.079823,0.079908,0.079950,0.080360,0.080765,0.081014,0.081144
1,20220830,0.082916,0.083771,0.082475,0.083766,0.084290,0.083840,0.083446,0.080556,0.080685,...,0.080763,0.079179,0.078912,0.079458,0.080223,0.080946,0.079696,0.081248,0.078490,0.081647
2,20220829,0.082916,0.083771,0.082855,0.083958,0.083714,0.083648,0.083446,0.080552,0.081281,...,0.081631,0.079568,0.078791,0.080008,0.081362,0.081654,0.079091,0.079553,0.075779,0.077334
3,20220826,0.083895,0.083771,0.082855,0.083958,0.083714,0.083648,0.083446,0.079833,0.079916,...,0.079291,0.078020,0.076979,0.078191,0.079485,0.079064,0.077100,0.075669,0.074616,0.073053
4,20220825,0.082916,0.083771,0.082855,0.084536,0.084674,0.084967,0.084399,0.081023,0.081969,...,0.082270,0.080699,0.079535,0.080619,0.081784,0.081358,0.079123,0.076941,0.072677,0.074795
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250,20210906,0.084890,0.081315,0.078494,0.084343,0.087362,0.088294,0.087993,0.087312,0.090632,...,0.090205,0.086675,0.083000,0.080238,0.077876,0.076419,0.069883,0.060714,0.051255,0.063523
251,20210903,0.084632,0.082661,0.078494,0.083766,0.087369,0.088274,0.087958,0.087688,0.090750,...,0.090734,0.087604,0.083793,0.080589,0.078133,0.076671,0.070166,0.060800,0.051348,0.063686
252,20210902,0.084890,0.082992,0.078494,0.084728,0.087374,0.088267,0.087994,0.087341,0.090467,...,0.090632,0.087085,0.083212,0.080228,0.077811,0.076395,0.069750,0.060164,0.051036,0.064613
253,20210901,0.084890,0.081808,0.076408,0.081840,0.087170,0.088463,0.088373,0.086892,0.090712,...,0.090375,0.087048,0.083406,0.080095,0.077586,0.076173,0.069116,0.059384,0.051135,0.063444


In [ ]:
dias = 180
curva_tasa=df
posicion = 1500


plazo = dias # REVISAR
ult_tasa = curva_tasa[str(plazo)][0]
df = pd.DataFrame({'Tasa':curva_tasa[str(plazo)]})
df['Ret_Tasa'] = (df['Tasa'].shift(1)/df['Tasa']) - 1
df['Ret_Tasa'][0] = 0
print(ult_tasa)
print(df)
df['Tasa_Act'] = ult_tasa*(1+df['Ret_Tasa'])

0.084098333
         Tasa  Ret_Tasa
0    0.084098  0.000000
1    0.084290 -0.002278
2    0.083714  0.006881
3    0.083714  0.000000
4    0.084674 -0.011338
..        ...       ...
250  0.087362 -0.019780
251  0.087369 -0.000078
252  0.087374 -0.000053
253  0.087170  0.002334
254  0.086786  0.004425

[255 rows x 2 columns]


# Bono M

In [ ]:
def bonoMyield(x, plazos, plazocupon, tfcupon, nominal, contratos):#valoración bono tasa fija
  
 N=int(plazos/plazocupon)+1   
 a=(1-(1+x*plazocupon/360)**(-N))/(plazocupon*x/360)
 p1=plazos-plazocupon*(N-1)   
 Vty=((contratos*nominal*tfcupon*plazocupon/360)*a+(contratos*nominal)/((1+x*plazocupon/360)**N))*(1+x*plazocupon/360)**(1-p1/plazocupon) 

 return Vty  

In [ ]:
x = bonoMyield(.0605,3600,182,0.065,100,1000)
x

104053.06525595774

R

In [ ]:
#CARGAMOS TODO LO QUE VAMOS A OCUPAR
%load_ext rpy2.ipython
!rm -rf RiesgosFinancieros
!git clone "https://github.com/BrandonRoberto/Tarea1-ARF"

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython
fatal: destination path 'Tarea1-ARF' already exists and is not an empty directory.


In [ ]:
%%R
## GENERAMOS LAS FUNCIONES QUE NECESITAREMOS:

# Función de interpolación de tasas por el método alamabrada
talamb=function(nodos,curva,plazos){
  n=max(ncol(plazos),1)
  m=max(ncol(nodos),1)
  TC=matrix(0,1,n)
  TL=matrix(0,1,n)
  TF=matrix(0,1,n)
  for (j in 1:n){
    i=1
    repeat
    {
      if(nodos[i]<= plazos[j] && plazos[j] <=nodos[i+1])
      {
        TC[j]=curva[i]
        TL[j]=curva[i+1]
        TF[j]=((((1+TL[j]*nodos[i+1]/360)/(1+TC[j]*nodos[i]/360))^((plazos[j]-nodos[i])/(nodos[i+1]-nodos[i]))*(1+TC[j]*nodos[i]/360))-1)*360/plazos[j]
        break
      }
      else if (plazos[j]<nodos[1])
      {
        TC[j]=curva[1]
        TL[j]=curva[1]
        TF[j]=curva[1]
        break
      }
      else if (plazos[j]>nodos[m])
      {
        TC[j]=curva[m]
        TL[j]=curva[m]
        TF[j]=curva[m]
        break
      }
      else
      {i=i+1}
    }
  }
  as.matrix(t(as.numeric(rbind(TF))))
}

# Función para diagonalizar un vector
diagv=function(x)	{
  n01=nrow(as.matrix(x))
  m01=ncol(as.matrix(x))
  dimmax=max(n01,m01)
  res=matrix(0,dimmax,dimmax)
  for (i in 1:dimmax){
    res[i,i]=x[i]
  }
  res
}

#Función de cuantil más cercano
equantile <- function(v,p=.5,ns=nrow(as.matrix(v))) 
{ 
  if ( !is.numeric(p) || any( p<0 | p>1) ) 
    stop("Percentil tiene que ser 0<=p<=1") 
  ranking <- order(v) 
  vw=matrix(0,ns,1)
  vw[1:ns]=seq(1/ns,ns)  
  sumw <- cumsum(vw[ranking]) 
  plist <- sumw / sumw[ length(sumw) ] 
  v [ ranking [ which.max( plist >= p ) ] ]  
} 

In [ ]:
%%R
#VaR con alisado

wquantile <- function(v,w=rep(1,length(v)),p=.5) 
{ 
  if ( !is.numeric(w) || length(v) != length(w) ) 
    stop("Los valores y los pesos tienen que tener misma longitud") 
  if ( !is.numeric(p) || any( p<0 | p>1) ) 
    stop("Percentil tiene que ser 0<=p<=1") 
  if ( min(w) < 0 ) stop("Los pesos tiene que ser mayores que 0") 
  ranking <- order(v) 
  sumw <- cumsum(w[ranking]) 
  plist <- sumw / sumw[ length(sumw) ] 
  v [ ranking [ which.max( plist >= p ) ] ]  
} 

#CVaR con alisado
wcvar <- function(v,w=rep(1,length(v)),p=.5) 
{ 
  if ( !is.numeric(w) || length(v) != length(w) ) 
    stop("Los valores y los pesos tienen que tener misma longitud") 
  if ( !is.numeric(p) || any( p<0 | p>1) ) 
    stop("Percentil tiene que ser 0<=p<=1") 
  if ( min(w) < 0 ) stop("Los pesos tiene que ser mayores que 0") 
  ranking <- order(v) 
  sumw <- cumsum(w[ranking]) 
  plist <- sumw / sumw[ length(sumw) ] 
  loss= v [ ranking [ which( plist <= p ) ] ]  
  esc=w [ ranking [ which( plist <= p ) ] ]  
  sum(loss*esc)/(sum(esc))
}

In [ ]:
%%R
#Definición de parámetros para valorar
fval=as.Date("20220831",format="%Y%m%d")
alpha = 0.98 #@param {type:"slider", min:0, max:1, step:0.01}
#poner 0 si se quiere interpolación lineal o 1 si se quiere tasa alambrada
itpl = 0 #@param ["0", "1"] {type:"raw"} 

# Bono M en R

In [ ]:
%%R
btasadesc="Tarea1-ARF/tasa_yield.txt"

itpl=0 #poner 0 si se quiere interpolación lineal o 1 si se quiere tasa alambrada

tf = cbind(0.065,0)
tfcupon = cbind(0.0605, 0)
plazos= cbind(3600, 0)
plazocupon = cbind(182, 0)
contratos= cbind(1, 0)
nominal = 1000

In [ ]:
%%R

#carga de datos
#carga de tasas de descuento
data1<-read.table(btasadesc)
n<-nrow(data1)
m_orig=ncol(data1)
X_orig=data.frame(data1[2:n,2:m_orig])
nodos=data.frame(data1[1,2:m_orig])
n=n-1
#Posición inicial
#interpolación de tasas y volatilidades
m=ncol(plazos)
X=matrix(0,n-1,m)
for (i in 1:(n-1))
{
X[i,]=if(itpl==0){approx(nodos,X_orig[i,],plazos)$y}else{talamb(nodos,X_orig[i,],plazos)}
}

In [ ]:
%%R

#Función de valoración por tasa yield
bonoMyield=function(x, plazos, plazocupon, tfcupon, nominal, contratos) #valoración bono tasa fija
{
x=x0  
N=as.integer(plazos/plazocupon)+1   
a=(1-(1+x*plazocupon/360)^(-N))/(plazocupon*x/360)
p1=plazos-plazocupon*(N-1)   
((contratos*nominal*tfcupon*plazocupon/360)*a+(contratos*nominal)/((1+x*plazocupon/360)^N))*(1+x*plazocupon/360)^(1-p1/plazocupon)   
}

x0=0.065 #tasas de descuento valor actual
V0_bonom = bonoMyield(x0,plazos, plazocupon, tfcupon, nominal, contratos)
V0_bonom[1,2] = 0
VT0_bonom=sum(V0_bonom)
print("Valor del Bono M")
VT0_bonom

[1] "Valor del Bono M"
[1] 973.9283


# CETE en R

In [ ]:
%%R
library(dplyr)
base=("Tarea1-ARF/tasa_guber.txt")
data_cetes<-read.table(base)
colnames(data_cetes)=data_cetes[1,]
data_cetes= data_cetes[-1,]
colnames(data_cetes)
data_cetes=data_cetes[,c('DATE', '180')]
data_cetes['Rendimientos']= (data_cetes[ "180"]/lag(data_cetes[ "180"], 1) )-1
data_cetes[is.na(data_cetes)]=0
data_cetes['Estimación escenario']= data_cetes[1,'180']*(1+data_cetes['Rendimientos'])
posicion=1500
plazo=180
data_cetes['Valor de la posicion']= (posicion*10)/(1+data_cetes['Estimación escenario']*plazo/360)
data_cetes[is.na(data_cetes)]=0
data_cetes['PL']=data_cetes['Valor de la posicion']-data_cetes[1,'Valor de la posicion']
print('Valor de la posición')
print(data_cetes[1,'Valor de la posicion'])


Attaching package: ‘dplyr’



    filter, lag



    intersect, setdiff, setequal, union




[1] "Valor de la posición"
[1] 14394.84


Bondes en R

In [ ]:
%%R
#install.packages("dplyr")
require(dplyr)
require(data.table)



Attaching package: ‘data.table’



    between, first, last




In [ ]:
%%R
btasadescst="Tarea1-ARF/tasa_guber_st.txt"
btasafondeo="Tarea1-ARF/tfondeo.txt"
plazos= cbind(707, 707) #Vencimiento del bono
plazocupon= cbind(28, 28) #plazos_bdm fijos de cada cupón
contratos = cbind(-1000, 0)
nominal=100

In [ ]:
%%R
#carga de datos
#carga de tasas de descuento
data1<-read.table(btasadescst)
n<-nrow(data1)
m_orig=ncol(data1)
X_orig=data.frame(data1[2:n,2:m_orig])
X1_orig=mutate(data.frame(data1[2:n,1:m_orig]), V1=as.Date(V1,format="%Y%m%d"))
nodos=data.frame(data1[1,2:m_orig])
n=n-1

data3<-read.table(btasadescst)
n3<-nrow(data3)
m3_orig=ncol(data3)
X3_orig=data.frame(data3[2:n3,2:m3_orig])
X3a_orig=mutate(data.frame(data1[2:n,1:m_orig]), V1=as.Date(V1,format="%Y%m%d"))
nodos3=data.frame(data3[1,2:m3_orig])
n3=n3-1

data2<-read.table(btasafondeo)
n2<-nrow(data2)
X2_orig=data.frame(data2[2:n2,1:2])
X2a_orig=mutate(X2_orig, V1=as.Date(V1,format="%Y%m%d"), V2=as.numeric(as.character(V2)))
tfh=seq(as.Date("20220331",format="%Y%m%d"), as.Date("20220331",format="%Y%m%d"), "days") #sucesión de dias para tasa fondeo
#tfhd=data.frame(ID=1:count(tfh),fecha=tfh)
tfhd=CJ(fecha=tfh)

In [ ]:
%%R
#Cruzar la sucesión de todos los días versus el de tasa de fondeo
tfhd=setDT(tfhd)[, join_date := tfh][order(-join_date)]
X2a_orig=setDT(X2a_orig)[, join_date := V1][order(-join_date)]
X2a_orig=X2a_orig[tfhd, on = .(join_date), roll = Inf]

In [ ]:
%%R
#Cruzar la sucesión de todos los días versus el de tasa de fondeo
fval=as.Date("20220831",format="%Y%m%d")
#buscar fecha de valuación en tfondeo
tf_act=X2a_orig[fecha==fval,]$V2/100
tf_int=X2a_orig[fecha<=fval & fecha>=(fval-plazocupon[1])]$V2/100

X1_orig=setDT(X1_orig)[, join_date := V1][order(-join_date)] #Para alinear con valor presente y tasa de fondeo.
X2_pr=X2a_orig[X1_orig, on = .(join_date)] #Se alinea la tasa de fondeo por fecha con la tasa de valor presente.
X2_pr

#ES POSIBLE QUE SI CORRES ESTA PARTE LOS VALORES DE V1 Y V2 SON NAN, PERO EN ESCENCIA LOS VALORES DE i.V1 Y i.V2 SON LOS MISMOS QUE V1 Y V2

       V1 V2  join_date fecha       i.V1       i.V2         V3         V4
  1: <NA> NA 2022-08-31  <NA> 2022-08-31 0.02536878 0.02537933 0.02541979
  2: <NA> NA 2022-08-30  <NA> 2022-08-30 0.02477881 0.02537933 0.02541979
  3: <NA> NA 2022-08-29  <NA> 2022-08-29 0.02477881 0.02537933 0.02553666
  4: <NA> NA 2022-08-26  <NA> 2022-08-26 0.02507143 0.02537933 0.02553666
  5: <NA> NA 2022-08-25  <NA> 2022-08-25 0.02477881 0.02537933 0.02553666
 ---                                                                     
251: <NA> NA 2021-09-06  <NA> 2021-09-06 0.02536878 0.02463528 0.02419262
252: <NA> NA 2021-09-03  <NA> 2021-09-03 0.02529165 0.02504289 0.02419262
253: <NA> NA 2021-09-02  <NA> 2021-09-02 0.02536878 0.02514324 0.02419262
254: <NA> NA 2021-09-01  <NA> 2021-09-01 0.02536878 0.02478464 0.02354982
255: <NA> NA 2021-08-31  <NA> 2021-08-31 0.02536878 0.02493193 0.02354982
             V5         V6         V7         V8         V9        V10
  1: 0.02605230 0.02680031 0.02759847 0.0

In [ ]:
%%R

plazocupon

     [,1] [,2]
[1,]   28   28


In [ ]:
%%R
matrix(0,1,sum(26))

     [,1] [,2] [,3] [,4] [,5] [,6] [,7] [,8] [,9] [,10] [,11] [,12] [,13] [,14]
[1,]    0    0    0    0    0    0    0    0    0     0     0     0     0     0
     [,15] [,16] [,17] [,18] [,19] [,20] [,21] [,22] [,23] [,24] [,25] [,26]
[1,]     0     0     0     0     0     0     0     0     0     0     0     0


In [ ]:
%%R      

m=ncol(plazos)


N=as.integer(plazos/plazocupon)+1 #número de cupones a pagar
VTplazos=matrix(0,1,sum(N)) #vector de todos los plazos de todos los contratos
contratosT=matrix(0,1,sum(N)) #vector de todos los contratos de todos los flujos de todos los contratos
nominalT=matrix(0,1,sum(N)) #vector de todos los nominales de todos los flujos de todos los contratos
plazocuponT=matrix(0,1,sum(N)) #vector de todos los plazoscupon de todos los flujos de todos los contratos
tasafijaT=matrix(0,1,sum(N)) #vector de tasas fijas de todos los flujos de todos los contratos
ulNomT=matrix(0,1,sum(N)) #vector de contratos a final de flujo


plazini=plazos-plazocupon*(N-1) #vector de plazos iniciales
ddv=plazocupon-plazini #dias trasncurridos del cupón vigente
tfcupon=matrix(0,1,m)  #El primero cupón de cada bono
tfcupondev=matrix(0,1,m) #cupón de los días devengados
tfcupgen=((1+tf_act/360)^(plazocupon[1])-1)*360/plazocupon[1] #el segundo al último cupón de todos los bonos
#calcula cupones de bonos


In [ ]:
%%R
tf_act

numeric(0)


In [ ]:
%%R
for (j in (1:m))
{
  tfcupondev[j]=(prod(1+tf_int[(1:ddv[j])]/360)-1)*360/ddv[j]
  tfcupon[j]=((1+tfcupondev[j]*ddv[j]/360)*(1+tf_act/360)^(plazocupon[1]-ddv[j])-1)*360/plazocupon[1]
}


for (j in (1:m))
{
  if (j==1)
  {
    VTplazos[,1:sum(N[1:j])]=seq(plazini[j],plazos[j], by=plazocupon[j])
    contratosT[,1:sum(N[1:j])]=seq(contratos[j],contratos[j])
    plazocuponT[,1:sum(N[1:j])]=seq(plazocupon[j],plazocupon[j])
    ulNomT[,sum(N[1:j])]=contratos[j]
    tasafijaT[,1]=tfcupon[j]
    tasafijaT[,2:sum(N[1:j])]=seq(tfcupgen,tfcupgen)
  }
  else
  {
    VTplazos[,(sum(N[1:j-1])+1):sum(N[1:j])]=seq(plazini[j],plazos[j], by=plazocupon[j])
    contratosT[,(sum(N[1:j-1])+1):sum(N[1:j])]=seq(contratos[j],contratos[j])					
    plazocuponT[,(sum(N[1:j-1])+1):sum(N[1:j])]=seq(plazocupon[j],plazocupon[j])
    tasafijaT[,(sum(N[1:j-1])+1)]=tfcupon[j]
    tasafijaT[,(sum(N[1:j-1])+2):sum(N[1:j])]=seq(tfcupgen,tfcupgen)
    ulNomT[,sum(N[1:j])]=contratos[j]
  }
}

Xvp=matrix(0,n,ncol(VTplazos))
Xst=matrix(0,n,ncol(VTplazos))

for (i in (1:n))
{
  Xvp[i,]=if(itpl==0){approx(nodos,X_orig[i,],VTplazos)$y}else{talamb(nodos,X_orig[i,],VTplazos)}
  Xst[i,]=if(itpl==0){approx(nodos3,X3_orig[i,],VTplazos)$y}else{talamb(nodos3,X3_orig[i,],VTplazos)}
  
}

head(Xvp)
head(Xst)


  replacement has length zero




Error in tfcupon[j] <- ((1 + tfcupondev[j] * ddv[j]/360) * (1 + tf_act/360)^(plazocupon[1] -  : 
  replacement has length zero


RInterpreterError: ignored

# VaR y CVaR Sin Alisado

In [ ]:
%%R
#DIMENSION DE TODOS LOS INSTRUMENTOS
#Son 3 instrumentos financieros 
n_if=matrix(0,2,1)
#n_if[1]=ncol(X_bd_ext) #bonde lo puse como comentario porque no corre
n_if[1]=1 #Cetes
n_if[2]=0 #Bonos M
#valor del portafolios

V0_port=cbind(VT0_bonom) #falta bondes y bonos

V0T_port=sum(V0_port)

In [ ]:
%%R
#INTEGRACIÓN DE TODOS LOS FACTORES DE RIESGO EN UNA MATRIZ
X_port=cbind( X[,1], X[,1]) #Factores de riesgo del portafolios de 8(9) instrumentos financieros
#Cambié la primer x por mayúsucla

#Cálculo de variaciones Delta_X DEL PORTAFOLIOS
DeltaX_port=as.matrix(log(X_port[1:(n-1)]/X_port[2:(n)]))
DeltaX_port[is.nan(DeltaX_port)] <- 0 #quitamos NaN
DeltaX_port[is.na(DeltaX_port)] <- 0 #quitamos Na
DeltaX_port[is.infinite(DeltaX_port)] <- 0 #quitamos Na


Ns=nrow(DeltaX_port) #Definimos número de escenarios históricos

DeltaX_s=DeltaX_port

In [ ]:
%%R
#Función de cuantil más cercano
equantile <- function(v,p=.5,ns=nrow(as.matrix(v))) 
{ 
  if ( !is.numeric(p) || any( p<0 | p>1) ) 
    stop("Percentil tiene que ser 0<=p<=1") 
  ranking <- order(v) 
  vw=matrix(0,ns,1)
  vw[1:ns]=seq(1/ns,ns)  
  sumw <- cumsum(vw[ranking]) 
  plist <- sumw / sumw[ length(sumw) ] 
  v [ ranking [ which.max( plist >= p ) ] ]  
} 

## CETE

In [ ]:
%%R
print('Valor de la posición')
V0_cetes= data_cetes[1,'Valor de la posicion']
print(V0_cetes)
VaR= quantile(data_cetes$PL,0.98 ) 
print('VaR')
print(VaR)
CVaR=mean(data_cetes$PL[data_cetes['PL']>VaR])
print('CVaR')
print(CVaR)

[1] "Valor de la posición"
[1] 14394.84
[1] "VaR"
     98% 
8.627165 
[1] "CVaR"
[1] 12.72356


## Bono M

No hay riesgo porque la tasa es fija. No se tiene riesgo, dado que desde el inicio ya sabemos como es que va evolucionando en cada periodo de tiempo.



## Bondes

Esto falta

# VaR y CVaR con alisado

## CETES

In [ ]:
###Cetes
%%R
wquantile <- function(v,w=rep(1,length(v)),p=.5){ 
  if ( !is.numeric(w) || length(v) != length(w) ) 
    stop("Los valores y los pesos tienen que tener misma longitud") 
  if ( !is.numeric(p) || any( p<0 | p>1) ) 
    stop("Percentil tiene que ser 0<=p<=1") 
  if ( min(w) < 0 ) stop("Los pesos tiene que ser mayores que 0") 
  ranking <- order(v) 
  sumw <- cumsum(w[ranking]) 
  plist <- sumw / sumw[ length(sumw) ] 
  v [ ranking [ which.max( plist >= p ) ] ]  
}
#CVaR con alisado
wcvar <- function(v,w=rep(1,length(v)),p=.5) { 
  if ( !is.numeric(w) || length(v) != length(w) ) 
    stop("Los valores y los pesos tienen que tener misma longitud") 
  if ( !is.numeric(p) || any( p<0 | p>1) ) 
    stop("Percentil tiene que ser 0<=p<=1") 
  if ( min(w) < 0 ) stop("Los pesos tiene que ser mayores que 0") 
  ranking <- order(v) 
  sumw <- cumsum(w[ranking]) 
  plist <- sumw / sumw[ length(sumw) ] 
  loss= v [ ranking [ which( plist < p ) ] ]  
  esc=w [ ranking [ which( plist < p ) ] ]  
  sum(loss*esc)/(sum(esc))
}

VaR_alisado=wquantile(data_cetes$PL)
print('VaR_alisado')
print(VaR_alisado)
CVaR_alisado=wcvar(data_cetes$PL)
print('CVaR_alisado')
print(CVaR_alisado)


[1] "VaR_alisado"
[1] 0
[1] "CVaR_alisado"
[1] -2.173817


## Bono M

No hay riesgo porque la tasa es fija. No se tiene riesgo, dado que desde el inicio ya sabemos como es que va evolucionando en cada periodo de tiempo.

## Bondes

In [ ]:
%%R
#install.packages("dplyr")
require(dplyr)
require(data.table)

In [ ]:
%%R
btasadescst="Tarea1-ARF/tasa_guber_st.txt"
btasafondeo="Tarea1-ARF/tfondeo.txt"
plazos= cbind(707, 707) #Vencimiento del bono
plazocupon= cbind(28, 28) #plazos_bdm fijos de cada cupón
contratos = cbind(-1000, 0)
nominal=100

In [ ]:
%%R
#carga de datos
#carga de tasas de descuento
data1<-read.table(btasadescst)
n<-nrow(data1)
m_orig=ncol(data1)
X_orig=data.frame(data1[2:n,2:m_orig])
X1_orig=mutate(data.frame(data1[2:n,1:m_orig]), V1=as.Date(V1,format="%Y%m%d"))
nodos=data.frame(data1[1,2:m_orig])
n=n-1

data3<-read.table(btasadescst)
n3<-nrow(data3)
m3_orig=ncol(data3)
X3_orig=data.frame(data3[2:n3,2:m3_orig])
X3a_orig=mutate(data.frame(data1[2:n,1:m_orig]), V1=as.Date(V1,format="%Y%m%d"))
nodos3=data.frame(data3[1,2:m3_orig])
n3=n3-1

data2<-read.table(btasafondeo)
n2<-nrow(data2)
X2_orig=data.frame(data2[2:n2,1:2])
X2a_orig=mutate(X2_orig, V1=as.Date(V1,format="%Y%m%d"), V2=as.numeric(as.character(V2)))
tfh=seq(as.Date("20220831",format="%Y%m%d"), as.Date("20220831",format="%Y%m%d"), "days") #sucesión de dias para tasa fondeo
#tfhd=data.frame(ID=1:count(tfh),fecha=tfh)
tfhd=CJ(fecha=tfh)

In [ ]:
%%R
#Cruzar la sucesión de todos los días versus el de tasa de fondeo
tfhd=setDT(tfhd)[, join_date := tfh][order(-join_date)]
X2a_orig=setDT(X2a_orig)[, join_date := V1][order(-join_date)]
X2a_orig=X2a_orig[tfhd, on = .(join_date), roll = Inf]

In [ ]:
%%R
#Cruzar la sucesión de todos los días versus el de tasa de fondeo
#buscar fecha de valuación en tfondeo
tf_act=X2a_orig[fecha==fval,]$V2/100
tf_int=X2a_orig[fecha<=fval & fecha>=(fval-plazocupon[1])]$V2/100

X1_orig=setDT(X1_orig)[, join_date := V1][order(-join_date)] #Para alinear con valor presente y tasa de fondeo.
X2_pr=X2a_orig[X1_orig, on = .(join_date)] #Se alinea la tasa de fondeo por fecha con la tasa de valor presente.
X2_pr

#ES POSIBLE QUE SI CORRES ESTA PARTE LOS VALORES DE V1 Y V2 SON NAN, PERO EN ESCENCIA LOS VALORES DE i.V1 Y i.V2 SON LOS MISMOS QUE V1 Y V2

             V1   V2  join_date      fecha       i.V1       i.V2         V3
  1: 2022-08-31 8.65 2022-08-31 2022-08-31 2022-08-31 0.02536878 0.02537933
  2:       <NA>   NA 2022-08-30       <NA> 2022-08-30 0.02477881 0.02537933
  3:       <NA>   NA 2022-08-29       <NA> 2022-08-29 0.02477881 0.02537933
  4:       <NA>   NA 2022-08-26       <NA> 2022-08-26 0.02507143 0.02537933
  5:       <NA>   NA 2022-08-25       <NA> 2022-08-25 0.02477881 0.02537933
 ---                                                                       
251:       <NA>   NA 2021-09-06       <NA> 2021-09-06 0.02536878 0.02463528
252:       <NA>   NA 2021-09-03       <NA> 2021-09-03 0.02529165 0.02504289
253:       <NA>   NA 2021-09-02       <NA> 2021-09-02 0.02536878 0.02514324
254:       <NA>   NA 2021-09-01       <NA> 2021-09-01 0.02536878 0.02478464
255:       <NA>   NA 2021-08-31       <NA> 2021-08-31 0.02536878 0.02493193
             V4         V5         V6         V7         V8         V9
  1: 0.02541979 0

In [ ]:
%%R      

m=ncol(plazos)


N=as.integer(plazos/plazocupon)+1 #número de cupones a pagar
VTplazos=matrix(0,1,sum(N)) #vector de todos los plazos de todos los contratos
contratosT=matrix(0,1,sum(N)) #vector de todos los contratos de todos los flujos de todos los contratos
nominalT=matrix(0,1,sum(N)) #vector de todos los nominales de todos los flujos de todos los contratos
plazocuponT=matrix(0,1,sum(N)) #vector de todos los plazoscupon de todos los flujos de todos los contratos
tasafijaT=matrix(0,1,sum(N)) #vector de tasas fijas de todos los flujos de todos los contratos
ulNomT=matrix(0,1,sum(N)) #vector de contratos a final de flujo


plazini=plazos-plazocupon*(N-1) #vector de plazos iniciales
ddv=plazocupon-plazini #dias trasncurridos del cupón vigente
tfcupon=matrix(0,1,m)  #El primero cupón de cada bono
tfcupondev=matrix(0,1,m) #cupón de los días devengados
tfcupgen=((1+tf_act/360)^(plazocupon[1])-1)*360/plazocupon[1] #el segundo al último cupón de todos los bonos
#calcula cupones de bonos
for (j in (1:m))
{
  tfcupondev[j]=(prod(1+tf_int[(1:ddv[j])]/360)-1)*360/ddv[j]
  tfcupon[j]=((1+tfcupondev[j]*ddv[j]/360)*(1+tf_act/360)^(plazocupon[1]-ddv[j])-1)*360/plazocupon[1]
}


for (j in (1:m))
{
  if (j==1)
  {
    VTplazos[,1:sum(N[1:j])]=seq(plazini[j],plazos[j], by=plazocupon[j])
    contratosT[,1:sum(N[1:j])]=seq(contratos[j],contratos[j])
    plazocuponT[,1:sum(N[1:j])]=seq(plazocupon[j],plazocupon[j])
    ulNomT[,sum(N[1:j])]=contratos[j]
    tasafijaT[,1]=tfcupon[j]
    tasafijaT[,2:sum(N[1:j])]=seq(tfcupgen,tfcupgen)
  }
  else
  {
    VTplazos[,(sum(N[1:j-1])+1):sum(N[1:j])]=seq(plazini[j],plazos[j], by=plazocupon[j])
    contratosT[,(sum(N[1:j-1])+1):sum(N[1:j])]=seq(contratos[j],contratos[j])					
    plazocuponT[,(sum(N[1:j-1])+1):sum(N[1:j])]=seq(plazocupon[j],plazocupon[j])
    tasafijaT[,(sum(N[1:j-1])+1)]=tfcupon[j]
    tasafijaT[,(sum(N[1:j-1])+2):sum(N[1:j])]=seq(tfcupgen,tfcupgen)
    ulNomT[,sum(N[1:j])]=contratos[j]
  }
}

Xvp=matrix(0,n,ncol(VTplazos))
Xst=matrix(0,n,ncol(VTplazos))

for (i in (1:n))
{
  Xvp[i,]=if(itpl==0){approx(nodos,X_orig[i,],VTplazos)$y}else{talamb(nodos,X_orig[i,],VTplazos)}
  Xst[i,]=if(itpl==0){approx(nodos3,X3_orig[i,],VTplazos)$y}else{talamb(nodos3,X3_orig[i,],VTplazos)}
  
}

head(Xvp)
head(Xst)


           [,1]       [,2]       [,3]       [,4]       [,5]       [,6]
[1,] 0.02537933 0.02547250 0.02576767 0.02606061 0.02629333 0.02652604
[2,] 0.02537933 0.02546752 0.02573481 0.02600220 0.02627254 0.02654288
[3,] 0.02537933 0.02557963 0.02582026 0.02605925 0.02625390 0.02644854
[4,] 0.02537933 0.02557963 0.02582026 0.02605925 0.02625390 0.02644854
[5,] 0.02537933 0.02559457 0.02591886 0.02623992 0.02647397 0.02670803
[6,] 0.02535696 0.02559955 0.02595172 0.02629969 0.02653419 0.02676869
           [,7]       [,8]       [,9]      [,10]      [,11]      [,12]
[1,] 0.02675876 0.02700429 0.02725260 0.02750091 0.02773433 0.02795811
[2,] 0.02681322 0.02704984 0.02727911 0.02750839 0.02774652 0.02799036
[3,] 0.02664318 0.02689701 0.02716370 0.02743040 0.02769517 0.02795871
[4,] 0.02664318 0.02689701 0.02716370 0.02743040 0.02769517 0.02795871
[5,] 0.02694208 0.02723575 0.02754239 0.02784902 0.02810864 0.02833784
[6,] 0.02700319 0.02731038 0.02763338 0.02795638 0.02822570 0.02846030
      

In [ ]:
%%R   

X_bd_tc=matrix(1,n,ncol(contratosT))*X2_pr$V2/100
X_bd_ext=cbind(X_bd_tc,as.matrix(Xvp),as.matrix(Xst))


bondeD=function(contratosT, nominal, tasafijaT, plazocuponT, VTplazos, Xvp, Xst, N)
{ 
  V0=matrix(0,1,nrow(as.matrix(N)))
  V0f=((((contratosT*(tasafijaT)*(plazocuponT/360))+ulNomT)/(1+(Xvp+Xst)*VTplazos/360)))*nominal
  V0f[1]=0
  V0f[27]=0
  for (j in (1:nrow(as.matrix(N))))
  {
    if(j==1)
    {
      V0[j]=sum(V0f[j:N[j]])
    }
    else
    {
      V0[j]=sum(V0f[(sum(N[1:j-1])+1):(sum(N[1:j]))])
    }
  }
  V0
} 

V0_bd=bondeD(contratosT, nominal, tasafijaT, plazocuponT, VTplazos, Xvp[1,], Xst[1,], N)
print("Valor del Bonde D")
print(V0_bd[1,1])

[1] "Valor del Bonde D"
[1] -105136.1


In [ ]:
VaR y CVaR Sin Alisado

SyntaxError: ignored

In [ ]:
%%R
#DIMENSION DE TODOS LOS INSTRUMENTOS
#Son 3 instrumentos financieros 
n_if=matrix(0,2,1)
n_if[1]=ncol(X_bd_ext) #bonde
n_if[1]=1 #Cetes
n_if[2]=0 #Bonos M
#valor del portafolios

V0_port=cbind(V0_bd, V0_cetes, VT0_bonom)

V0T_port=sum(V0_port)
print(V0T_port)

[1] -89767.29


In [ ]:
%%R
X_bd_tc

         [,1]   [,2]   [,3]   [,4]   [,5]   [,6]   [,7]   [,8]   [,9]  [,10]
  [1,] 0.0865 0.0865 0.0865 0.0865 0.0865 0.0865 0.0865 0.0865 0.0865 0.0865
  [2,]     NA     NA     NA     NA     NA     NA     NA     NA     NA     NA
  [3,]     NA     NA     NA     NA     NA     NA     NA     NA     NA     NA
  [4,]     NA     NA     NA     NA     NA     NA     NA     NA     NA     NA
  [5,]     NA     NA     NA     NA     NA     NA     NA     NA     NA     NA
  [6,]     NA     NA     NA     NA     NA     NA     NA     NA     NA     NA
  [7,]     NA     NA     NA     NA     NA     NA     NA     NA     NA     NA
  [8,]     NA     NA     NA     NA     NA     NA     NA     NA     NA     NA
  [9,]     NA     NA     NA     NA     NA     NA     NA     NA     NA     NA
 [10,]     NA     NA     NA     NA     NA     NA     NA     NA     NA     NA
 [11,]     NA     NA     NA     NA     NA     NA     NA     NA     NA     NA
 [12,]     NA     NA     NA     NA     NA     NA     NA     NA     NA     NA

In [ ]:
%%R
#INTEGRACIÓN DE TODOS LOS FACTORES DE RIESGO EN UNA MATRIZ
X_port=cbind( X_bd_ext, X[,1]) #Factores de riesgo del portafolios de 8(9) instrumentos financieros

#Cálculo de variaciones Delta_X DEL PORTAFOLIOS
DeltaX_port=as.matrix(log(X_port[1:(n-1)]/X_port[2:(n)]))
DeltaX_port[is.nan(DeltaX_port)] <- 0 #quitamos NaN
DeltaX_port[is.na(DeltaX_port)] <- 0 #quitamos Na
DeltaX_port[is.infinite(DeltaX_port)] <- 0 #quitamos Na


Ns=nrow(DeltaX_port) #Definimos número de escenarios históricos

DeltaX_s=DeltaX_port

In [ ]:
%%R
#Función de cuantil más cercano
equantile <- function(v,p=.5,ns=nrow(as.matrix(v))) 
{ 
  if ( !is.numeric(p) || any( p<0 | p>1) ) 
    stop("Percentil tiene que ser 0<=p<=1") 
  ranking <- order(v) 
  vw=matrix(0,ns,1)
  vw[1:ns]=seq(1/ns,ns)  
  sumw <- cumsum(vw[ranking]) 
  plist <- sumw / sumw[ length(sumw) ] 
  v [ ranking [ which.max( plist >= p ) ] ]  
} 

In [ ]:
%%R
print('Valor de la posición')
print(data_cetes[1,'Valor de la posicion'])
VaR= quantile(data_cetes$PL,0.98 ) 
print('VaR')
print(VaR)
CVaR=mean(data_cetes$PL[data_cetes['PL']>VaR])
print('CVaR')
print(CVaR)


[1] "Valor de la posición"
[1] 14394.84
[1] "VaR"
     98% 
8.627165 
[1] "CVaR"
[1] 12.72356


In [ ]:
%%R
#sum(n_if[1:2])
DeltaX_s[,1]


  [1] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 [38] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 [75] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
[112] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
[149] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
[186] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
[223] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


In [ ]:
## Bondes D

%%R

#Cálculo de matriz de pérdidas y ganancias BONDES
#dimensión
m=N  #PASO CLAVE
X_s_bd=matrix(0,Ns,n_if[2]) #Factores de riesgo simulados con base en DeltaX_s x0*(1+Delta_Xs)   #PASO CLAVE
V_bd=matrix(0,Ns,m)
Vfr1_bd=matrix(0,Ns,m)
Vfr2_bd=matrix(0,Ns,m)
Vfr3_bd=matrix(0,Ns,m)

DeltaX_s_bd=DeltaX_s[,sum(n_if[1:1],1):sum(n_if[1:2])]    #PASO CLAVE
x0_bd=X_bd_ext[1,]    #PASO CLAVE

for (i in 1:Ns)
{
  X_s_bd[i,]=x0_bd*exp(DeltaX_s_bd[i])
  #PASO CLAVE
  V_bd[i,]=bondeD(contratosT, nominal, X_s_bd[i,1], plazocuponT, VTplazos, X_s_bd[i,(n_if[2]/3+1):(n_if[2]*2/3)], X_s_bd[i,(n_if[2]*2/3+1):(n_if[2])], N,ddv)
  #PASO CLAVE
  Vfr1_bd[i,]=bondeD(contratosT, nominal, X_s_bd[i,1], plazocuponT, VTplazos, x0_bd[(n_if[2]/3+1):(n_if[2]*2/3)], x0_bd[(n_if[2]*2/3+1):(n_if[2])], N,ddv)
  #PASO CLAVE
  Vfr2_bd[i,]=bondeD(contratosT, nominal, x0_bd[1], plazocuponT, VTplazos, X_s_bd[i,(n_if[2]/3+1):(n_if[2]*2/3)], x0_bd[(n_if[2]*2/3+1):(n_if[2])], N,ddv)
  #PASO CLAVE
  Vfr3_bd[i,]=bondeD(contratosT, nominal, x0_bd[1], plazocuponT, VTplazos, x0_bd[(n_if[2]/3+1):(n_if[2]*2/3)], X_s_bd[i,(n_if[2]*2/3+1):(n_if[2])], N,ddv)
} 


print("Valor del bonde D")
print(V0_bd[1,1])
print("Valor en riesgo del bonde D")
print(VaRCont_bd[1,1])
print("Valor en Riesgo Condicional de bondes D")
print(CVaRCont_bd[1,1])


  subscript out of bounds


 




Error in DeltaX_s[, sum(n_if[1:1], 1):sum(n_if[1:2])] : 
  subscript out of bounds


RInterpreterError: ignored

# Futuros

100 contratos de compra de futuros de peso d ́olar con un strike de 20.83 vencimiento de 5 d ́ıas, y 50 contratos
de venta de futuros del IPC con strike de 49525 vencimiento de 53 d ́ıas.

In [ ]:
%load_ext rpy2.ipython
!rm -rf RiesgosFinancieros
!git clone "https://github.com/BrandonRoberto/Tarea1-ARF"
#!git clone "https://github.com/pikey-msc/RiesgosFinancieros/"
#!git clone "https://github.com/ArathReyes/Risk-Management/"


The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython
fatal: destination path 'Tarea1-ARF' already exists and is not an empty directory.


In [ ]:
def Forward(tipo,M,S,K,vencimiento,tx,ty):
  if tipo == 'Cambio':
    precio = M*((S*((1+(tx*vencimiento/360))/(1+(ty*vencimiento/360)))- K)/(1+(tx*vencimiento/360)))
    return precio
  elif tipo == 'Indice':
    precio = M*(( S*(1+((tx-ty)*vencimiento/360))   - K)/(1+(tx*vencimiento/360)))
    return precio
  else:
    return 'Tipo de Forward incorrecto'
def intlin(x,X,Y):
    n = len(X)
    for i in range(n-1):
        if (X[i]<= x <= X[i+1]):
            a = Y[i+1] - Y[i]
            b = X[i+1] - X[i]
            return Y[i] + (x - X[i])*(a/b)
    return "No es posible interpolar"
class Riesgo_Forward:

  def __init__(self):
    self.posiciones = None
    self.strikes = None
    self.precios = None
    self.VaR = None
    self.CVaR = None
    self.summary = None
    self.precios = None
    self.alisado = None
    self.alpha = None

  def compute(self,M,S,K,vencimientos, tx,ty,alpha = 0.98, alisado = False):
    l = []
    for i in range(2):
      string = 'cambio' if i == 0 else 'indice'
      aux1 = pd.DataFrame()
      aux2 = pd.DataFrame()
      aux1['DATE'] = tx[i]['DATE']
      aux2['DATE'] = ty[i]['DATE']

      if (str(vencimientos[i]) in tx[i].columns):
        aux1['tx_'+string] = tx[i][str(vencimientos[i])]
      else:
        X = list(tx[i].columns)
        X.remove('DATE')
        X = [int(i) for i in X]
        aux1['tx_'+string] = [intlin(vencimientos[i],X, tx[i].iloc[j,1:].values) for j in range(tx[i].shape[0])]

      if (str(vencimientos[i]) in ty[i].columns):
        aux2['ty_'+string] = ty[i][str(vencimientos[i])]
      else:
        X = list(ty[i].columns)
        X.remove('DATE')
        X = [int(i) for i in X]
        aux2['ty_'+string] = [intlin(vencimientos[i],X, ty[i].iloc[j,1:].values) for j in range(ty[i].shape[0])]
      c = aux1.merge(aux2,on = 'DATE',how = 'inner')
      l.append(c)
    df = l[0].merge(l[1],on = 'DATE',how = 'inner')
    df = df.merge(S, how = 'inner',on = 'DATE')
    df = df.set_index('DATE')
    X = df.copy()
    X0 = X.head(1)
    DeltaX = (df.shift(1)/df)-1
    DeltaX.iloc[0,:] = 0
    X_s = X0.values*(1+DeltaX)

    Y_s = pd.DataFrame()
    Y_s['Cambio'] = [Forward('Cambio',M[0],X_s['USDMXN=X'][i],K[0],vencimientos[0],X_s['tx_cambio'][i],X_s['ty_cambio'][i])for i in range(X_s.shape[0])]
    Y_s['Indice'] = [Forward('Indice',M[0],X_s['^MXX'][i],K[1],vencimientos[1],X_s['tx_indice'][i],X_s['ty_indice'][i])for i in range(X_s.shape[0])]
    Y0 = Y_s.head(1)
    Y0 = Y0.values[0]
    PL = Y_s - Y0
    PL['Total']=PL.sum(axis=1)

    VaR_C=[]

    if alisado:
      c = 1
      PL.dropna(inplace = True)
      #PL = PL.iloc[::-1]
      PL["Esc_SA"] = ExponentialWeighted(0.05,PL.shape[0])
      col_aux = list(PL.columns)
      col_aux.remove("Esc_SA")
      vector_VaR = [VaR_CA(PL.iloc[:,i],PL.Esc_SA,alpha) for i in range(PL.shape[1]-1)]
      VaR = pd.Series(vector_VaR,index = col_aux)
    else:
      c = 0
      vector_VaR = [PL.iloc[:,i].quantile(alpha) for i in range(2)]
      VaR = pd.Series(vector_VaR,index = ['Cambio','Indice'])
      VaR = PL.quantile(alpha,numeric_only = True)
    
    for i in range(0,PL.shape[1]-c):
      VC=CVaR(PL.iloc[:,i],VaR[i])
      VaR_C.append(VC)

    self.summary = pd.DataFrame({"VaR":VaR})
    self.summary['CVaR'] = VaR_C

    self.posiciones = M
    self.strikes = K
    self.VaR = self.summary[['VaR']]
    self.CVaR = self.summary[['CVaR']]
    self.alpha = alpha
    self.alisado = alisado
    self.precios = list(Y0)

    return

miForward = Riesgo_Forward()

Libor=pd.read_table("Tarea1-ARF/tasa_libor.txt")
Libor = Libor.rename(columns = {'DATE ':'DATE'})
Libor['DATE'] =Libor['DATE'].apply(lambda x: str(x))
Libor['DATE'] = Libor['DATE'].apply(lambda x: datetime(int(x[:4]), int(x[4:6]), int(x[6:])))
nombre = list(Libor.columns)
nombre.remove('DATE')
Libor[nombre] = Libor[nombre]/100

Guber= pd.read_table("Tarea1-ARF/tasa_guber.txt")
Guber = Guber.rename(columns = {'DATE ':'DATE'})
Guber['DATE'] =Guber['DATE'].apply(lambda x: str(x))
Guber['DATE'] = Guber['DATE'].apply(lambda x: datetime(int(x[:4]), int(x[4:6]), int(x[6:])))


div=pd.read_table("Tarea1-ARF/tasa_dividendos.txt")
div = div.rename(columns = {'DATE ':'DATE'})
div['DATE'] =div['DATE'].apply(lambda x: str(x))
div['DATE'] = div['DATE'].apply(lambda x: datetime(int(x[:4]), int(x[4:6]), int(x[6:])))
nombre = list(div.columns)
nombre.remove('DATE')
div[nombre] = div[nombre]/100

S = pd.read_table("Tarea1-ARF/tasas_IPC.txt") 
S['Close']= S['Close'].replace("N/E", np.NaN)
S= S.dropna()
S['Close'] = pd.to_numeric(S['Close'])
S = S.rename(columns = {'Date':'DATE','Close':'^MXX'})
S['DATE'] =S['DATE'].apply(lambda x: str(x))
S['DATE'] = S['DATE'].apply(lambda x: datetime(int(x[:4]), int(x[4:6]), int(x[6:])))

Y = pd.read_table("Tarea1-ARF/tasa_spot.txt")
Y = Y.rename(columns = {'date':'DATE','precio':'USDMXN=X'})
Y['DATE'] =Y['DATE'].apply(lambda x: str(x))
Y['DATE'] = Y['DATE'].apply(lambda x: datetime(int(x[:4]), int(x[4:6]), int(x[6:])))




S = S.merge(Y, on = 'DATE',how='inner')

tx = [Guber,Guber]
ty=[Libor,div]

#M = [500,50]
#K =[19.83,49525]
#K = [20.5,49525]
#vencimientos = [360,53]

K = [20.83,49525]
M = [100,50]
vencimientos = [5,53]

## Sin Alisado

In [ ]:
miForward.compute(M,S,K,vencimientos,tx,ty)


In [ ]:
miForward.precios

[29.94886146355832, -447314.33771317534]

In [ ]:
miForward.summary

,VaR,CVaR
Cambio,29.712031,50.246304
Indice,87267.207748,97769.046719
Total,87265.096336,97762.079719


## Con Alisado

In [ ]:
miForward.compute(M,S,K,vencimientos,tx,ty,alisado = True)

In [ ]:
miForward.summary

,VaR,CVaR
Cambio,99.399117,99.399117
Indice,79203.974973,90527.523530
Total,79216.098969,90525.910572


# Swap

Un swap largo con nocional de 16 millones pagando tasa fija de 6.6% anual (cada 28 d ́ıas) y recibiendo tasa
flotante de la curva TIIE vencimiento de 588 d ́ıas, un swap corto con nocional de 12 millones pagando tasa
variable de la TIIE y recibiendo fija de 5.9% vencimiento de 270 d ́ıas.

In [ ]:
def InterpolacionLineal(x,X,Y):
    n = len(X)
    for i in range(n-1):
        if X[i]<= x <= X[i+1]:
            a = Y[i+1] - Y[i]
            b = (X[i+1] - X[i]).days
            return Y[i] + (x - X[i]).days*(a/b)
    return "No es posible interpolar"

class InterestRateSwap:
    
    def __init__(self):
        self.precio = None
        self.n_cupones = None
        self.nocional = None
        self.posicion = None
        self.convencion = None
        self.fija = None
        self.flotante = None
        self.descuentos = None
        self.summary = None
        
    def compute(self, fval, M, n, fix_rate, flotante,calen_flotante, descuentos, calen_desc, 
                plazos, tau,  payer = False, act_360 = True):
        """
        M: Nocional
        n: # de cupones
        fix_rate: Tasa fija de la pata larga del IRS
        flotante: Curva de tasas de mercado (la que encuentras en BANXICO) de la pata corta
        calen_flotante: Las fechas dadas por la curva de flotante
        descuentos: Curca de tasas de descuento de mercado para valuar el IRS
        calen_desc: Las fechas dadas por la curva de descuentos
        plazos : Lista de plazo del i -ésimo cupón (habiles)
        tau: Plazo en dias del i-ésimo cupón. (naturales)
        payer: Dummy; 1 si paga fija, 0 si paga flotante
        act_360: Dummy; tipo de convención 1 si act/360, 0 si act/365
        """

        spot = fval
        
        if act_360:
            conv = 360
        else:
            conv = 365
            
        aux = pd.DataFrame({"Cupon":list(range(1,n+1)), "Tau":tau})
        aux["Desde Spot"] = aux["Tau"].cumsum()
        aux["Fecha"] = aux["Desde Spot"].apply(lambda x: timedelta(x) + spot)
                
        df = pd.DataFrame({"Cupon":list(range(1,n+1)), "Plazo": plazos, "Tau": tau})
                
        df["Fechas"] =  aux["Fecha"]
        df["Flotante"] = df["Fechas"].apply(InterpolacionLineal, args=(calen_flotante, flotante))
        df["Descuentos"]=df["Fechas"].apply(InterpolacionLineal, args=(calen_desc, descuentos))
        
        # Calculo de tasas forward
        df['Forward_Flotante'] = (conv/(aux['Desde Spot']-aux['Desde Spot'].shift(1)))*(((1+(df['Flotante']*aux['Desde Spot']/conv))/(1+(df['Flotante'].shift(1)*aux['Desde Spot'].shift(1)/conv))) - 1)
        df['Forward_Flotante'][0] = df['Flotante'][0]
        
        df["Sumando"] = ((df["Forward_Flotante"]-fix_rate)*df["Plazo"]/conv) / (1 + (df["Descuentos"]*df["Tau"]/conv))
        
        self.precio =  M*((-1)**payer)*df["Sumando"].sum()
        self.n_cupones = n
        self.nocional = M
        self.posicion = "Payer" if payer else "Receiver"
        self.convencion = conv
        self.fija = fix_rate
        self.flotante = df[["Fechas","Forward_Flotante"]]
        self.descuentos = df[["Fechas", "Descuentos"]]
        self.summary = df
        
        return
    
    def plots(self):
        Bool = True
        while Bool:
            print("\n")
            choice = input("Si deseas ver las tasas de descuento teclea 'Descuentos', si deseas\
                                ver las tasas flotantes teclea 'Tasas' o si deseas salir escribe '0':\n")
            if choice == 'Descuentos' or choice =='Tasas' or choice == '0':
                Bool = False
            else:
                print('\nTu elección no es válida')
        if choice == 'Descuentos':
            sns.set_style('darkgrid')
            sns.set_palette('tab10')
            plt.figure(figsize = (12,8))
            ax = sns.lineplot(x = self.summary['Fechas'], y = self.summary['Descuentos'])
            ax.set_title("Curva de Descuentos",fontsize = '25')
            plt.show()
            
            return
        elif choice == 'Tasas':
            sns.set_style('darkgrid')
            sns.set_palette('tab10')
            plt.figure(figsize = (12,8))
            ax = sns.lineplot(x = self.summary['Fechas'], y = self.summary['Forward_Flotante'], color = 'red')
            ax.set_title("Curva de Tasas Forward Flotantes",fontsize = '25')
            plt.show()
            
            return
        else:
            return

In [ ]:
class Riesgo_IRS:

  def __init__(self):
    self.posiciones = None
    self.tasas_fijas = None
    self.vencimientos = None
    self.dias_bono = None
    self.VaR = None
    self.CVaR = None
    self.alpha = None
    self.summary = None
    self.ordenado = None
    self.alisado = None
    self.precios = None
    
  def compute(self, fval, tasas_fijas, cupon_l, descuento_l,cupon_c,descuento_c,\
              vencimientos, dias_bono, posiciones, alpha,alisado =False):
    """
    a
    """
    calen_flotante_l = cupon_l.columns
    calen_flotante_l = list(calen_flotante_l)
    calen_flotante_l.remove('DATE')
    calen_desc_l = descuento_l.columns
    calen_desc_l = list(calen_desc_l)
    calen_desc_l.remove('DATE')


    calen_flotante_c = cupon_c.columns
    calen_flotante_c = list(calen_flotante_c)
    calen_flotante_c.remove('DATE')
    calen_desc_c = descuento_c.columns
    calen_desc_c = list(calen_desc_c)
    calen_desc_c.remove('DATE')

    # ya
    n_l = int(vencimientos[0]/dias_bono[0])
    if n_l == (vencimientos[0]/dias_bono[0]):
        plazos_l = [dias_bono[0]]*n_l
        tau_l = plazos_l
    else:
        plazos_l = [vencimientos[0]-n_l*dias_bono[0]] + [dias_bono[0]]*(n_l-1)
        tau_l = plazos_l

    n_c = int(vencimientos[1]/dias_bono[1])
    if n_c == (vencimientos[1]/dias_bono[1]):
        plazos_c = [dias_bono[1]]*n_c
        tau_c = plazos_c
    else:
        plazos_c = [vencimientos[1]-n_c*dias_bono[1]] + [dias_bono[1]]*(n_c-1)
        tau_c = plazos_c



    n_cupl = cupon_l.shape[1] - 1
    n_descl = descuento_l.shape[1] - 1
    n_cupc = cupon_c.shape[1] - 1
    n_descc = descuento_c.shape[1] - 1

    # 
    X = cupon_l.merge(descuento_l,how = 'inner',on = 'DATE')
    X = X.merge(cupon_c,how = 'inner',on = 'DATE')
    X = X.merge(descuento_c,how = 'inner',on = 'DATE')
    X.set_index('DATE', inplace=True)
    X0 = X.head(1)
    DeltaX = (X.shift(1)/X)-1
    DeltaX.iloc[0,:] = 0
    X_s = X0.values*(1+DeltaX)

    Y_s = pd.DataFrame()

    cup_l = X_s.iloc[:,:n_cupl]
    desc_l = X_s.iloc[:,n_cupl:n_cupl+n_descl]
    cup_c = X_s.iloc[:,n_cupl+n_descl:n_cupl+n_descl+n_cupc]
    desc_c = X_s.iloc[:,n_cupl+n_descl+n_cupc:n_cupl+n_descl+n_cupc+n_descc]

    # Largo
    largo = []
    for i in range(len(cup_l)):
      fval_aux = cup_l.index[i]
      flotante =  cup_l.iloc[i,:].values/100
      flotante = list(flotante)
      descuentos = desc_l.iloc[i,:].values/100
      descuentos = list(descuentos)
      calen_flotante_l_aux = [fval_aux + timedelta(int(i)) for i in calen_flotante_l]
      calen_desc_l_aux = [fval_aux + timedelta(int(i)) for i in calen_desc_l]
      irs = InterestRateSwap()
      # fval vs fval_aux
      irs.compute(fval_aux, posiciones[0], n_l,tasas_fijas[0] , flotante,calen_flotante_l_aux, descuentos, calen_desc_l_aux, plazos_l, tau_l)
      largo.append(irs.precio)

    # Corto
    corto = []
    for i in range(len(cup_c)):
      fval_aux = cup_c.index[i]
      flotante =  cup_c.iloc[i,:].values/100
      flotante = list(flotante)
      descuentos = desc_c.iloc[i,:].values/100
      descuentos = list(descuentos)
      calen_flotante_c_aux = [fval_aux + timedelta(int(i)) for i in calen_flotante_c]
      calen_desc_c_aux = [fval_aux + timedelta(int(i)) for i in calen_desc_c]
      irs = InterestRateSwap()
      # fval vs fval_aux
      irs.compute(fval_aux, posiciones[1], n_c, tasas_fijas[1] , flotante,calen_flotante_c_aux, descuentos, calen_desc_c_aux, plazos_c, tau_c)
      corto.append(irs.precio)

    Y_s['IRS_l'] = largo
    Y_s['IRS_c'] = corto
    #print(Y_s)
    posiciones = pd.Series(posiciones,index = ['IRS_l','IRS_c'])
    Y0 = Y_s.head(1)
    Y0 = Y0.values[0]
    self.precios = Y0
    #print(Y0)
    PL = (Y_s-Y0)*posiciones
    PL['Total']=PL.sum(axis=1)
    #print(PL)
    VaR_C=[]

    if alisado:
      c = 1
      PL.dropna(inplace = True)
      #PL = PL.iloc[::-1]
      PL["Esc_SA"] = ExponentialWeighted(0.05,PL.shape[0])
      col_aux = list(PL.columns)
      col_aux.remove("Esc_SA")
      vector_VaR = [VaR_CA(PL.iloc[:,i],PL.Esc_SA,alpha) for i in range(PL.shape[1]-1)]
      VaR = pd.Series(vector_VaR,index = col_aux)
    else:
      c = 0
      VaR=PL.quantile(alpha, numeric_only=False)


    for i in range(0,PL.shape[1]-c):
      VC=CVaR(PL.iloc[:,i],VaR[i])
      VaR_C.append(VC)
    
    self.summary = pd.DataFrame({"VaR":VaR})
    self.summary['CVaR'] = VaR_C

    self.posiciones = {'Largo':posiciones[0],'Corto':posiciones[1]}
    self.tasas_fijas = {'Largo':tasas_fijas[0],'Corto':tasas_fijas[1]}
    self.vencimientos = {'Largo':vencimientos[0],'Corto':vencimientos[1]}
    self.dias_bono = {'Largo':dias_bono[0],'Corto':dias_bono[1]}
    self.VaR = self.summary[['VaR']]
    self.CVaR = self.summary[['CVaR']]
    self.alpha = alpha
    self.alisado = alisado
    self.precios = Y0
    return 


In [ ]:
# Actualizar datos con los de la tarea una vez nos den las nuevas tasas
fval = datetime(2022,8,31) # l,c

tasas_fijas = [0.066, 0.059]

cupon_l = pd.read_csv("Tarea1-ARF/tasa_DIRS_SW_OP.txt", delimiter = "\t")
#cupon_l = pd.read_csv("/tasa_DIRS_SW_OP.txt", delimiter = "\t")
cupon_l = cupon_l.rename(columns = {'DATE ':'DATE'})
cupon_l['DATE'] =cupon_l['DATE'].apply(lambda x: str(x))
cupon_l['DATE'] = cupon_l['DATE'].apply(lambda x: datetime(int(x[:4]), int(x[4:6]), int(x[6:])))
cupon_c  = cupon_l

descuento_l = pd.read_csv("Tarea1-ARF/tasa_TIIE_SW_OP.txt", delimiter = "\t")
#descuento_l = pd.read_csv("/tasa_TIIE_SW_OP.txt", delimiter = "\t")
descuento_l = descuento_l.rename(columns = {'DATE ':'DATE'})
descuento_l['DATE'] =descuento_l['DATE'].apply(lambda x: str(x))
descuento_l['DATE'] = descuento_l['DATE'].apply(lambda x: datetime(int(x[:4]), int(x[4:6]), int(x[6:])))
descuento_c = descuento_l

vencimientos = [588,270]

dias_bono = [28,28]

posiciones  = [16000000, -12000000]
alpha = 0.98


In [ ]:
miIRS= Riesgo_IRS()

## Sin Alisado

In [ ]:
miIRS.compute(fval, tasas_fijas, cupon_l, descuento_l,cupon_c,descuento_c,\
              vencimientos, dias_bono, posiciones, alpha,False)

In [ ]:
miIRS.summary

## Con Alisado

In [ ]:
miIRS.compute(fval, tasas_fijas, cupon_l, descuento_l,cupon_c,descuento_c,\
              vencimientos, dias_bono, posiciones, alpha,True)

In [ ]:
miIRS.summary

In [ ]:
print(miIRS.posiciones)
print(miIRS.tasas_fijas)
print(miIRS.vencimientos)
print(miIRS.dias_bono)
print(miIRS.precios)

# Opciones

Dos opciones europeas, una de tasa de inter ́es larga call con strike de 5.8 % vencimiento de 1700 d ́ıas 1000
contratos, y otra put larga de tasa de inter ́es con strike de 6.0 % de la tiie nocional de 500 contratos vencimiento
700 d ́ıas. (se valuar ́a basado en las superficies de volatilidad si es in o out the money y en las curvas de tasa de
inter ́es TIIE o Pagar ́es).

In [ ]:
%%R
#require(quantmod)
install.packages("quantmod")
library(quantmod)
# require(data.table)
install.packages("data.table")
library(data.table)

install.packages("Deriv")
library(Deriv)


library(dplyr)
con = gzcon(url('https://github.com/systematicinvestor/SIT/raw/master/sit.gz', 'rb'))
source(con)
close(con)

In [ ]:
%%R
#FUNCIÓN DE INTERPOLACIÓN ALAMBRADA

talamb=function(nodos,curva,plazos) #función de interpolación de tasas por el método alamabrada
{
  n=max(ncol(plazos),1)
  m=max(ncol(nodos),1)
  TC=matrix(0,1,n)
  TL=matrix(0,1,n)
  TF=matrix(0,1,n)
  for (j in 1:n)
  {
    i=1
    repeat
    {
      if(nodos[i]<= plazos[j] && plazos[j] <=nodos[i+1])
      {
        TC[j]=curva[i]
        TL[j]=curva[i+1]
        TF[j]=((((1+TL[j]*nodos[i+1]/360)/(1+TC[j]*nodos[i]/360))^((plazos[j]-nodos[i])/(nodos[i+1]-nodos[i]))*(1+TC[j]*nodos[i]/360))-1)*360/plazos[j]
        break
      }
      else if (plazos[j]<nodos[1])
      {
        TC[j]=curva[1]
        TL[j]=curva[1]
        TF[j]=curva[1]
        break
      }
      else if (plazos[j]>nodos[m])
      {
        TC[j]=curva[m]
        TL[j]=curva[m]
        TF[j]=curva[m]
        break
      }
      else
      {i=i+1}
    }
  }
  as.matrix(t(as.numeric(rbind(TF))))
}

#funciones necesarias
diagv=function(x)			#función para diagonalizar un vector
{
  n01=nrow(as.matrix(x))
  m01=ncol(as.matrix(x))
  dimmax=max(n01,m01)
  res=matrix(0,dimmax,dimmax)
  for (i in 1:dimmax)
  {
    res[i,i]=x[i]
  }
  res
}


diagm=function(x)		#función para diagonalizar una matriz en una de mayor dimensión
{
  n01=nrow(as.matrix(x))
  m01=ncol(as.matrix(x))
  if(m01>=n01 )
  {
    res=matrix(0,m01,m01)
    for (i in 1:(m01/n01))
    {
      z=(i-1)*n01
      for(j in 1:n01)
      {
        for(k in 1:n01)
        {
          res[z+k,z+j]=x[k,z+j]
        }
      }
    }
    res
  }
  else
  {
    "no es matriz cuadrada"
  }
}

#Función de cuantil más cercano
equantile <- function(v,p=.5,ns=nrow(as.matrix(v))) 
{ 
  if ( !is.numeric(p) || any( p<0 | p>1) ) 
    stop("Percentil tiene que ser 0<=p<=1") 
  ranking <- order(v) 
  vw=matrix(0,ns,1)
  vw[1:ns]=seq(1/ns,ns)  
  sumw <- cumsum(vw[ranking]) 
  plist <- sumw / sumw[ length(sumw) ] 
  v [ ranking [ which.max( plist >= p ) ] ]  
} 

In [ ]:
%%R

#MEDIDAS DE RIESGO CON ALISADO

#VaR con alisado
wquantile <- function(v,w=rep(1,length(v)),p=.5) 
{ 
  if ( !is.numeric(w) || length(v) != length(w) ) 
    stop("Los valores y los pesos tienen que tener misma longitud") 
  if ( !is.numeric(p) || any( p<0 | p>1) ) 
    stop("Percentil tiene que ser 0<=p<=1") 
  if ( min(w) < 0 ) stop("Los pesos tiene que ser mayores que 0") 
  ranking <- order(v) 
  sumw <- cumsum(w[ranking]) 
  plist <- sumw / sumw[ length(sumw) ] 
  v [ ranking [ which.max( plist >= p ) ] ]  
} 

#CVaR con alisado
wcvar <- function(v,w=rep(1,length(v)),p=.5) 
{ 
  if ( !is.numeric(w) || length(v) != length(w) ) 
    stop("Los valores y los pesos tienen que tener misma longitud") 
  if ( !is.numeric(p) || any( p<0 | p>1) ) 
    stop("Percentil tiene que ser 0<=p<=1") 
  if ( min(w) < 0 ) stop("Los pesos tiene que ser mayores que 0") 
  ranking <- order(v) 
  sumw <- cumsum(w[ranking]) 
  plist <- sumw / sumw[ length(sumw) ] 
  loss= v [ ranking [ which( plist <= p ) ] ]  
  esc=w [ ranking [ which( plist <= p ) ] ]  
  sum(loss*esc)/(sum(esc))
} 


In [ ]:
%%R
fval=as.Date("20220831",format="%Y%m%d") #Fecha de valoración
itpl=0 #poner 0 si se quiere interpolación lineal o 1 si se quiere tasa alambrada
alpha=0.98 #Nivel de confianza para obtener estimaciones de riesgo
nh=3660 #número de días de historia
#yext=1 #si se usa la historia de internet o fija

btasadesc_oir="Tarea1-ARF/tasa_TIIE_SW_OP.txt"
btasaspot_oir="Tarea1-ARF/tasa_DIRS_SW_OP.txt"
bvolspot_oir="Tarea1-ARF/tvoltiie_opc.txt"
plazos_oir=cbind(1700, 700) #T-t
pr_oir=28 #plazo de referencia
dct_oir=360 #d_base
cp_oir=cbind(1,0) #si es call (cap) o put (floor)
K_oir=cbind( 0.058, 0.06)
contratos_oir=cbind(1000, 500)
nominal_oir=1 
cs_oir=1 #1 si es continua la tasa 0 si es simple

In [ ]:
%%R

#CARGA DE DATOS PARA OPCIONES DE TASA DE INTERÉS
  #carga de datos
  #carga de rho	
  data1<-read.table(btasadesc_oir) 
  n<-nrow(data1)
  m1_orig_oir=ncol(data1)
  x1_orig_oir=data.table(mutate(data1[2:n,1:m1_orig_oir],Date=as.Date(V1,format="%Y%m%d")))
  x1_orig_oir=x1_orig_oir%>%select(-V1)
  nodos1_oir=data.frame(data1[1,2:m1_orig_oir])
  
  #data1[1:3,]
  #carga de tasas spot
  data2<-read.table(btasaspot_oir)
  n<-nrow(data2)
  m2_orig_oir=ncol(data2)
  x2_orig_oir=data.table(mutate(data2[2:n,1:m2_orig_oir],Date=as.Date(V1,format="%Y%m%d")))
  x2_orig_oir=x2_orig_oir%>%select(-V1)
  nodos2_oir=data.frame(data2[1,2:m2_orig_oir])
  #carga de volatilidades de spot
  data3<-read.table(bvolspot_oir)
  n<-nrow(data3)
  m3_orig_oir=ncol(data3)
  x3_orig_oir=data.table(mutate(data3[2:n,1:m3_orig_oir],Date=as.Date(V1,format="%Y%m%d")))
  x3_orig_oir=x3_orig_oir%>%select(-V1)
  
  nodos3_oir=data.frame(data3[1,2:m3_orig_oir])

In [ ]:
%%R
#INTERSECCIÓN DE FECHAS DE LAS OPCIONES    
  
lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvp=data.table(Date=as.Date(x1_orig_oir[x2_orig_oir,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, guber, st (bonde), libor, fwd, spot, equity or index, swap cupón, opciones ir spot, opc ir vp
lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol=data.table(Date=as.Date(lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvp[x3_orig_oir,on=.(Date),nomatch=0]$Date)) #Fechas acciones, equity, guber, st (bonde), libor, fwd, spot, equity or index, swap cupón, opciones ir spot, opc ir vp, opc ir vol


In [ ]:
%%R
head(lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvp)
head(lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol)
#Hace match con las fechas de las tasas (factores de riesgo)

In [ ]:
%%R
n=nrow(lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol) #Historia de todos

#historia de opciones
x1_orig_oir=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[x1_orig_oir,on=.(Date),nomatch=0][order(-Date)]
x1_orig_oir=x1_orig_oir%>%select(-Date)/100
x2_orig_oir=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[x2_orig_oir,on=.(Date),nomatch=0][order(-Date)]
x2_orig_oir=x2_orig_oir%>%select(-Date)/100
x3_orig_oir=lin_gub_bmybdst_flibfwdspind_swcupvp_oirsvpvol[x3_orig_oir,on=.(Date),nomatch=0][order(-Date)]
x3_orig_oir=x3_orig_oir%>%select(-Date)

In [ ]:
%%R


##opciones de tasa de interés, con inicio el día de la valuación CÁLCULO
#Posición inicial

#interpolación de tasas y volatilidades
m=ncol(plazos_oir)
x1=matrix(0,n,m)
x2tc=matrix(0,n,m)
x2tl=matrix(0,n,m)
x2=matrix(0,n,m)
x3=matrix(0,n,m)
for (i in 1:(n))
{
  x1[i,]=if(itpl==0){approx(nodos1_oir,x1_orig_oir[i,],plazos_oir,rule=2)$y}else{talamb(nodos1_oir,x1_orig_oir[i,],plazos_oir)}
  x2tc[i,]=if(itpl==0){approx(nodos2_oir,x2_orig_oir[i,],plazos_oir,rule=2)$y}else{talamb(nodos2_oir,x2_orig_oir[i,],plazos_oir)}
  x2tl[i,]=if(itpl==0){approx(nodos2_oir,x2_orig_oir[i,],(plazos_oir+pr_oir),rule=2)$y}else{talamb(nodos2_oir,x2_orig_oir[i,],(plazos_oir+pr_oir))}
  x3[i,]=if(itpl==0){approx(nodos3_oir,x3_orig_oir[i,],plazos_oir,rule=2)$y}else{talamb(nodos3_oir,x3_orig_oir[i,],plazos_oir)}
  x2[i,]=((1+x2tl[i,]*(plazos_oir+pr_oir)/360)/(1+x2tc[i,]*(plazos_oir)/360)-1)*360/pr_oir
}

#Se toma la primera fila de las tasas, ie la tasa más actual para calcular la valoracion
x01=x1[1,] #tasas de descuento
x02=x2[1,] #tasas spot
x03=x3[1,] #volatilidades

X_oir=cbind(x1,x2,x3)

opctint = function(d,S,K_oir,vol,t,cp_oir,cs_oir,pr_oir,dct_oir)	#función de una opción europea
{
  d1=if(cs_oir==1){(log(S/K_oir)+vol^2*t/(365*2))*(1/(vol*sqrt(t/365)))}else{(log(S/K_oir)+vol^2*t/(360*2))*(1/(vol*sqrt(t/360)))}
  d2=if(cs_oir==1){(log(S/K_oir)-vol^2*t/(365*2))*(1/(vol*sqrt(t/365)))}else{(log(S/K_oir)-vol^2*t/(360*2))*(1/(vol*sqrt(t/360)))}
  vp=if(cs_oir==1){log(1+d*t/360)*365/t}else{d}  
  (if(cs_oir==1){(S*pnorm(d1*(-1)^cp_oir)-K_oir*pnorm(d2*(-1)^cp_oir))*(exp(-vp*t/365))*(-1)^cp_oir}else{(S*pnorm(d1*(-1)^cp_oir)-K_oir*pnorm(d2*(-1)^cp_oir))*(1/(1+vp*t/360))*(-1)^cp_oir})*(pr_oir/dct_oir)/(1+S*pr_oir/dct_oir)
}

#Usa la funcion para valuar las opciones
V0_oir=opctint(x01,x02,K_oir,x03,plazos_oir,cp_oir,cs_oir,pr_oir,dct_oir)*contratos_oir*nominal_oir #Valor del portafolio
V0_oir


In [ ]:
%%R
#DIMENSION DE TODOS LOS INSTRUMENTOS
n_if=matrix(0,1,1)
n_if[1]=ncol(X_oir) #fwd tdc

#valor del portafolio

V0_port=cbind(V0_oir)
V0T_port=sum(V0_port)


#INTEGRACIÓN DE TODOS LOS FACTORES DE RIESGO EN UNA MATRIZ
X_port=cbind(X_oir) #Factores de riesgo del portafolios de 2 opciones y 3 factores de riesgo

#Tasas que nos interesan 
#print(head(X_port))


#Cálculo de variaciones Delta_X DEL PORTAFOLIOS
DeltaX_port=as.matrix(log(X_port[1:(n-1),]/X_port[2:(n),]))
DeltaX_port[is.nan(DeltaX_port)] <- 0 #quitamos NaN
DeltaX_port[is.na(DeltaX_port)] <- 0 #quitamos Na
DeltaX_port[is.infinite(DeltaX_port)] <- 0 #quitamos Na

Ns=nrow(DeltaX_port) #Definimos número de escenarios históricos
DeltaX_s=DeltaX_port


## VaR sin Alisado

In [ ]:
%%R
m=ncol(plazos_oir)  #PASO CLAVE
X_s_oir=matrix(0,Ns,n_if[1]) #Factores de riesgo simulados con base en DeltaX_s x0*(1+Delta_Xs) #PASO CLAVE
V_oir=matrix(0,Ns,m)
Vfr1_oir=matrix(0,Ns,m)
Vfr2_oir=matrix(0,Ns,m)
Vfr3_oir=matrix(0,Ns,m)
PG_oir=matrix(0,Ns,m) #Pèrdidas y ganancias
PGfr1_oir=matrix(0,Ns,m)
PGfr2_oir=matrix(0,Ns,m)
PGfr3_oir=matrix(0,Ns,m)
PGT_oir=matrix(0,Ns,1)
PGfr1T_oir=matrix(0,Ns,1)
PGfr2T_oir=matrix(0,Ns,1)
PGfr3T_oir=matrix(0,Ns,1)

DeltaX_s_oir=DeltaX_s  #PASO CLAVE
x0_oir=X_oir[1,] #PASO CLAVE

for (i in 1:Ns)
{
  X_s_oir[i,]=x0_oir*exp(DeltaX_s_oir[i,])
  #PASO CLAVE
  V_oir[i,]=  opctint(X_s_oir[i,(1:(n_if[1]/3))],X_s_oir[i,((n_if[1]/3+1):(n_if[1]/3*2))],K_oir,X_s_oir[i,((n_if[1]*2/3+1):(n_if[1]))],plazos_oir,cp_oir,cs_oir,pr_oir,dct_oir)*contratos_oir*nominal_oir #Valor del portafolio  
  #PASO CLAVE
  Vfr1_oir[i,]=opctint(X_s_oir[i,(1:(n_if[1]/3))],x0_oir[((n_if[1]/3+1):(n_if[1]/3*2))],K_oir,x0_oir[((n_if[1]*2/3+1):(n_if[1]))],plazos_oir,cp_oir,cs_oir,pr_oir,dct_oir)*contratos_oir*nominal_oir #Valor del portafolio  
  #PASO CLAVE
  Vfr2_oir[i,]=opctint(x0_oir[(1:(n_if[1]/3))],X_s_oir[i,((n_if[1]/3+1):(n_if[1]/3*2))],K_oir,x0_oir[((n_if[1]*2/3+1):(n_if[1]))],plazos_oir,cp_oir,cs_oir,pr_oir,dct_oir)*contratos_oir*nominal_oir #Valor del portafolio  
  #PASO CLAVE
  Vfr3_oir[i,]=opctint(x0_oir[(1:(n_if[1]/3))],x0_oir[((n_if[1]/3+1):(n_if[1]/3*2))],K_oir,X_s_oir[i,((n_if[1]*2/3+1):(n_if[1]))],plazos_oir,cp_oir,cs_oir,pr_oir,dct_oir)*contratos_oir*nominal_oir #Valor del portafolio  
  PG_oir[i,]=V_oir[i,]-V0_oir
  PGfr1_oir[i,]=Vfr1_oir[i,]-V0_oir
  PGfr2_oir[i,]=Vfr2_oir[i,]-V0_oir
  PGfr3_oir[i,]=Vfr3_oir[i,]-V0_oir
  PGT_oir[i,]=sum(PG_oir[i,])
  PGfr1T_oir[i,]=sum(PGfr1_oir[i,])
  PGfr2T_oir[i,]=sum(PGfr2_oir[i,])
  PGfr3T_oir[i,]=sum(PGfr3_oir[i,])
}



#VaR por posición
VaRCont_oir=matrix(0,1,m)
VaRfr1_oir=matrix(0,1,m)
VaRfr2_oir=matrix(0,1,m)
VaRfr3_oir=matrix(0,1,m)
CVaRCont_oir=matrix(0,1,m)
CVaRfr1_oir=matrix(0,1,m)
CVaRfr2_oir=matrix(0,1,m)
CVaRfr3_oir=matrix(0,1,m)
for (i in (1:m))
{
  VaRCont_oir[i]=equantile(PG_oir[,i],1-alpha,Ns)
  VaRfr1_oir[i]=equantile(PGfr1_oir[,i],1-alpha,Ns)
  VaRfr2_oir[i]=equantile(PGfr2_oir[,i],1-alpha,Ns)
  VaRfr3_oir[i]=equantile(PGfr3_oir[,i],1-alpha,Ns)
  CVaRfr1_oir[i]= mean(merge(which(PGfr1_oir[,i]<VaRfr1_oir[i]),cbind(seq(1,Ns),PGfr1_oir[,i]), by.x=1,by.y=1)[,2])
  CVaRfr2_oir[i]= mean(merge(which(PGfr2_oir[,i]<VaRfr2_oir[i]),cbind(seq(1,Ns),PGfr2_oir[,i]), by.x=1,by.y=1)[,2])
  CVaRfr3_oir[i]= mean(merge(which(PGfr3_oir[,i]<VaRfr3_oir[i]),cbind(seq(1,Ns),PGfr3_oir[,i]), by.x=1,by.y=1)[,2])
  CVaRCont_oir[i]= mean(merge(which(PG_oir[,i]<VaRCont_oir[i]),cbind(seq(1,Ns),PG_oir[,i]), by.x=1,by.y=1)[,2])
}


#VaR Total
VaRTotal_oir=equantile(PGT_oir,1-alpha,Ns)
CVaRTotal_oir= mean(merge(which(PGT_oir<VaRTotal_oir),cbind(seq(1,Ns),PGT_oir), by.x=1,by.y=1)[,2])
VaRTotalfr1_oir=equantile(PGfr1T_oir,1-alpha,Ns)
CVaRTotalfr1_oir= mean(PGfr1T_oir[which(PGfr1T_oir<VaRTotalfr1_oir),])
VaRTotalfr2_oir=equantile(PGfr2T_oir,1-alpha,Ns)
CVaRTotalfr2_oir= mean(PGfr2T_oir[which(PGfr2T_oir<VaRTotalfr2_oir),])
VaRTotalfr3_oir=equantile(PGfr3T_oir,1-alpha,Ns)
CVaRTotalfr3_oir= mean(PGfr3T_oir[which(PGfr2T_oir<VaRTotalfr2_oir),])


print("Valor del portafolio")
print(V0_oir)
print("VaR por contrato")
print(VaRCont_oir)
print("VaR por el factor de riesgo tasas de descuento por contrato")
print(VaRfr1_oir)
print("VaR por el factor de riesgo tasas spot por contrato")
print(VaRfr2_oir)
print("VaR por el factor de riesgo volatilidades por contrato")
print(VaRfr3_oir)
print("VaR por el factor de riesgo tasas de descuento total")
print(VaRTotalfr1_oir)
print("VaR por el factor de riesgo tasas spot 2 total")
print(VaRTotalfr2_oir)
print("VaR por el factor de riesgo volatilidades total")
print(VaRTotalfr3_oir)
print("VaR Total")
print(VaRTotal_oir)

print("CVaR por contrato")
print(CVaRCont_oir)
print("CVaR por el factor de riesgo tasas de descuento por contrato")
print(CVaRfr1_oir)
print("CVaR por el factor de riesgo tasas spot por contrato")
print(CVaRfr2_oir)
print("CVaR por el factor de riesgo volatilidades por contrato")
print(CVaRfr3_oir)
print("CVaR por el factor de riesgo tasas de descuento total")
print(CVaRTotalfr1_oir)
print("CVaR por el factor de riesgo tasas spot total")
print(CVaRTotalfr2_oir)
print("CVaR por el factor de riesgo volatilidades total")
print(CVaRTotalfr3_oir)
print("CVaR Total")
print(CVaRTotal_oir)


## VaR con Alisado

In [ ]:
%%R
w0 <- 0.05
lambda =uniroot(function(x) w0*(1-x^(n))/(1-x)-1, c(0,0.99), tol = 1e-28)$root


#generamos la función que genera "n" escenarios con base en w0 y lambda
genera_esc=function(lamda,w0,n)
{
  p_esc=matrix(0,n,1)
  for (i in (1:n))
  {
    p_esc[i]=w0*lambda^(i-1)
  }
  p_esc
}

p_esc=genera_esc(lambda,w0,Ns)

In [ ]:
%%R
m=ncol(plazos_oir)
VaRCACont_oir=matrix(0,1,m)
VaRCAfr1_oir=matrix(0,1,m)
VaRCAfr2_oir=matrix(0,1,m)
VaRCAfr3_oir=matrix(0,1,m)
CVaRCACont_oir=matrix(0,1,m)
CVaRCAfr1_oir=matrix(0,1,m)
CVaRCAfr2_oir=matrix(0,1,m)
CVaRCAfr3_oir=matrix(0,1,m)
for (i in (1:m))
{
  VaRCACont_oir[i]=wquantile(PG_oir[,i],p_esc,1-alpha)
  VaRCAfr1_oir[i]=wquantile(PGfr1_oir[,i],p_esc,1-alpha)
  VaRCAfr2_oir[i]=wquantile(PGfr2_oir[,i],p_esc,1-alpha)
  VaRCAfr3_oir[i]=wquantile(PGfr3_oir[,i],p_esc,1-alpha)
  CVaRCAfr1_oir[i]= wcvar(PGfr1_oir[,i],p_esc,1-alpha)
  CVaRCAfr2_oir[i]= wcvar(PGfr2_oir[,i],p_esc,1-alpha)
  CVaRCAfr3_oir[i]= wcvar(PGfr3_oir[,i],p_esc,1-alpha)
  CVaRCACont_oir[i]= wcvar(PG_oir[,i],p_esc,1-alpha)
}


#VaRCA Total
VaRCATotal_oir=wquantile(PGT_oir,p_esc,1-alpha)
CVaRCATotal_oir= wcvar(PGT_oir,p_esc,1-alpha)
VaRCATotalfr1_oir=wquantile(PGfr1T_oir,p_esc,1-alpha)
CVaRCATotalfr1_oir= wcvar(PGfr1T_oir,p_esc,1-alpha)
VaRCATotalfr2_oir=wquantile(PGfr2T_oir,p_esc,1-alpha)
CVaRCATotalfr2_oir= wcvar(PGfr2T_oir,p_esc,1-alpha)
VaRCATotalfr3_oir=wquantile(PGfr3T_oir,p_esc,1-alpha)
CVaRCATotalfr3_oir= wcvar(PGfr3T_oir,p_esc,1-alpha)

print("Valor del portafolio")
print(V0_oir)
print("VaRCA por contrato")
print(VaRCACont_oir)
print("VaRCA por el factor de riesgo tasas de descuento por contrato")
print(VaRCAfr1_oir)
print("VaRCA por el factor de riesgo tasas spot por contrato")
print(VaRCAfr2_oir)
print("VaRCA por el factor de riesgo volatilidades por contrato")
print(VaRCAfr3_oir)
print("VaRCA por el factor de riesgo tasas de descuento total")
print(VaRCATotalfr1_oir)
print("VaRCA por el factor de riesgo tasas spot total")
print(VaRCATotalfr2_oir)
print("VaRCA por el factor de riesgo volatilidades total")
print(VaRCATotalfr3_oir)
print("VaRCA Total")
print(VaRCATotal_oir)

print("CVaRCA por contrato")
print(CVaRCACont_oir)
print("CVaRCA por el factor de riesgo tasas de descuento por contrato")
print(CVaRCAfr1_oir)
print("CVaRCA por el factor de riesgo tasas spot por contrato")
print(CVaRCAfr2_oir)
print("CVaRCA por el factor de riesgo volatilidades por contrato")
print(CVaRCAfr3_oir)
print("CVaRCA por el factor de riesgo tasas de descuento total")
print(CVaRCATotalfr1_oir)
print("CVaRCA por el factor de riesgo tasas spot total")
print(CVaRCATotalfr2_oir)
print("CVaRCA por el factor de riesgo volatilidades total")
print(CVaRCATotalfr3_oir)
print("CVaRCA Total")
print(CVaRCATotal_oir)

# Código de Chris

In [ ]:
%load_ext rpy2.ipython
!rm -rf RiesgosFinancieros
!git clone "https://github.com/BrandonRoberto/Tarea1-ARF"

In [ ]:
%%R
#require(quantmod)
install.packages("quantmod")
library(quantmod)
#require(data.table)
install.packages("data.table")
library(data.table)
#require("PerformanceAnalytics")
install.packages("PerformanceAnalytics")
library("PerformanceAnalytics")

install.packages("Deriv")
library(Deriv)


library(dplyr)
con = gzcon(url('https://github.com/systematicinvestor/SIT/raw/master/sit.gz', 'rb'))
source(con)
close(con)

In [ ]:
%%R
#FUNCIÓN DE INTERPOLACIÓN ALAMBRADA

talamb=function(nodos,curva,plazos) #función de interpolación de tasas por el método alamabrada
{
  n=max(ncol(plazos),1)
  m=max(ncol(nodos),1)
  TC=matrix(0,1,n)
  TL=matrix(0,1,n)
  TF=matrix(0,1,n)
  for (j in 1:n)
  {
    i=1
    repeat
    {
      if(nodos[i]<= plazos[j] && plazos[j] <=nodos[i+1])
      {
        TC[j]=curva[i]
        TL[j]=curva[i+1]
        TF[j]=((((1+TL[j]*nodos[i+1]/360)/(1+TC[j]*nodos[i]/360))^((plazos[j]-nodos[i])/(nodos[i+1]-nodos[i]))*(1+TC[j]*nodos[i]/360))-1)*360/plazos[j]
        break
      }
      else if (plazos[j]<nodos[1])
      {
        TC[j]=curva[1]
        TL[j]=curva[1]
        TF[j]=curva[1]
        break
      }
      else if (plazos[j]>nodos[m])
      {
        TC[j]=curva[m]
        TL[j]=curva[m]
        TF[j]=curva[m]
        break
      }
      else
      {i=i+1}
    }
  }
  as.matrix(t(as.numeric(rbind(TF))))
}


#funciones necesarias
diagv=function(x)			#función para diagonalizar un vector
{
  n01=nrow(as.matrix(x))
  m01=ncol(as.matrix(x))
  dimmax=max(n01,m01)
  res=matrix(0,dimmax,dimmax)
  for (i in 1:dimmax)
  {
    res[i,i]=x[i]
  }
  res
}

#función de cuantil más cercano
equantile <- function(v,p=.5,ns=nrow(as.matrix(v))) 
{ 
  if ( !is.numeric(p) || any( p<0 | p>1) ) 
    stop("Percentil tiene que ser 0<=p<=1") 
  ranking <- order(v) 
  vw=matrix(0,ns,1)
  vw[1:ns]=seq(1/ns,ns)  
  sumw <- cumsum(vw[ranking]) 
  plist <- sumw / sumw[ length(sumw) ] 
  v [ ranking [ which.max( plist >= p ) ] ]  
} 


wquantile <- function(v,w=rep(1,length(v)),p=.5) 
{ 
  if ( !is.numeric(w) || length(v) != length(w) ) 
    stop("Los valores y los pesos tienen que tener misma longitud") 
  if ( !is.numeric(p) || any( p<0 | p>1) ) 
    stop("Percentil tiene que ser 0<=p<=1") 
  if ( min(w) < 0 ) stop("Los pesos tiene que ser mayores que 0") 
  ranking <- order(v) 
  sumw <- cumsum(w[ranking]) 
  plist <- sumw / sumw[ length(sumw) ] 
  v [ ranking [ which.max( plist >= p ) ] ]  
}


#CVaR con alisado
wcvar <- function(v,w=rep(1,length(v)),p=.5) 
{ 
  if ( !is.numeric(w) || length(v) != length(w) ) 
    stop("Los valores y los pesos tienen que tener misma longitud") 
  if ( !is.numeric(p) || any( p<0 | p>1) ) 
    stop("Percentil tiene que ser 0<=p<=1") 
  if ( min(w) < 0 ) stop("Los pesos tiene que ser mayores que 0") 
  ranking <- order(v) 
  sumw <- cumsum(w[ranking]) 
  plist <- sumw / sumw[ length(sumw) ] 
  loss= v [ ranking [ which( plist < p ) ] ]  
  esc=w [ ranking [ which( plist < p ) ] ]  
  sum(loss*esc)/(sum(esc))
}

## Definición de Parámetros para valorar

In [ ]:
%%R
fval=as.Date("20200813",format="%Y%m%d") #Fecha de valoración
itpl=0 #poner 0 si se quiere interpolación lineal o 1 si se quiere tasa alambrada
alpha=0.98 #Nivel de confianza para obtener estimaciones de riesgo
#setwd(direc)	


#CETES
base="Tarea1-ARF/tasa_guber.txt"

#BONDES D
btasadescst="Tarea1-ARF/tasa_guber_st.txt"
btasafondeo="Tarea1-ARF/tfondeo.txt"
plazos_bdm=cbind( 707,	707) #Vencimiento del bono
plazocupon_bdm=cbind( 28,	28) #plazos_bdm fijos de cada cupón
contratos_bdm=cbind(-1000,0) #posición invertida
nominal_bdm=100

#FORWARDS TDC
bext="Tarea1-ARF/tasa_libor.txt"
bdom="Tarea1-ARF/tasa_fwd.txt"
SymbolsFX_ftdc<-c("USDMXN=X" ) #tienen que ir en orden alfabético
plazos_fwd=cbind( 5,53	)
contratos_fwd=cbind(100,50)
kst_fwd=cbind(20.83, 49525)
nominal_fwd=1
yext=1 #si se carga información de yahoo en la fecha definida por fval o SymbolsFX, en caso contrario se utiliza información que se tendrá que cargar de tasas_spot.txt
trlib=1 #1 si la curva libor viene a 182 0 si no.


#SWAP
btasadesc_sw="Tarea1-ARF/tasa_TIIE_SW_OP.txt"
btasacupvar_sw="Tarea1-ARF/tasa_DIRS_SW_OP.txt"
tasafija_sw=cbind(0.066,0.059) #se establece la tasa fija a pagar para cada swap
plazos_sw=cbind(588,270) #se establece el número de días que vivirá el swap
plazocupon_sw=cbind(28,28) #se establece el número de días que se pagará cada cupón
contratos_sw=cbind(16000000,-12000000) #se establece el número de contratos_sw de cada swap
nominal_sw=cbind(1,1) #se establece el nominal_sw de cada swap
por_sw=cbind(0,0) #se establece 0 si se paga tasa fija y 1 si se paga tasa variable

#OPCIONES
btasadesc_oir="Tarea1-ARF/tasa_TIIE_SW_OP.txt"
btasaspot_oir="Tarea1-ARF/tasa_DIRS_SW_OP.txt"
bvolspot_oir="Tarea1-ARF/tvoltiie_opc.txt"
plazos_oir=cbind( 39,	115,	153,	245,	360) #T-t
pr_oir=28 #plazo de referencia  
dct_oir=360 #d_base
cp_oir=cbind(1,0,0,0,0) #si es call (cap) o put (floor)
K_oir=cbind( 0.07, 0.085, 0.078, 0.085, 0.08)
contratos_oir=cbind(2200, -2900, 2900, -460, 5000)
nominal_oir=1
cs_oir=1 #1 si es continua la tasa 0 si es simple


In [ ]:
%%R



#LEER DATOS DE CETES #CARGA DE DATOS DE BONO CUPÓN CERO
data<-read.table(base)
n<-nrow(data)
m_gov=ncol(data)
#x_orig_gov=data.frame(data[2:n,1:m_gov])
x_orig_gov=as.data.table(mutate(data[2:n,1:m_gov],Date=as.Date(V1,format="%Y%m%d")))
x_orig_gov=x_orig_gov%>%select(-V1)
nodos_gov=data.frame(data[1,2:m_gov])


##CARGA DE DATOS DE BONDE D
#carga de datos
#carga de tasas de descuento
data1<-read.table(base)
n<-nrow(data1)
m_bd=ncol(data1)
##X_orig_bd=as.data.table(mutate(data1[2:n,1:m_tybm],Date=as.Date(V1,format="%Y%m%d")))
#X_orig_bd%>%select(-V1)

X1_orig=mutate(data.frame(data1[2:n,1:m_bd]), V1=as.Date(V1,format="%Y%m%d"))
print(head(X1_orig[,1:4]))
#nodos=data.frame(data1[1,2:m_bd])
n=n-1

data3<-read.table(btasadescst)
n3<-nrow(data3)
m3_bd=ncol(data3)
X3_orig_bd=as.data.table(mutate(data3[2:n,1:m3_bd],Date=as.Date(V1,format="%Y%m%d")))
X3_orig_bd=X3_orig_bd%>%select(-V1)

X3a_orig=mutate(data.frame(data1[2:n,1:m_bd]), V1=as.Date(V1,format="%Y%m%d"))
nodos3_bd=data.frame(data3[1,2:m3_bd])
n3=n3-1

data2<-read.table(btasafondeo)
n2<-nrow(data2)
X2_orig_bd=data.frame(data2[2:n2,1:2])
print(head(X2_orig_bd))
X2_orig_bd=mutate(X2_orig_bd, V1=as.Date(V1,format="%Y%m%d"), Date=as.Date(V1,format="%Y%m%d"), V2=as.numeric(as.character(V2)))
tfh=seq(min(X2_orig_bd$V1), max(X2_orig_bd$V1), "days") #sucesión de dias para tasa fondeo
tfhd=data.frame(ID=1:count(tfh),fecha=tfh)

#Cruzar la sucesión de todos los días versus el de tasa de fondeo
tfhd=setDT(tfhd)[, Date := tfh][order(-Date)]
X2_orig_bd=setDT(X2_orig_bd)[, Date := V1][order(-Date)]
# rolling join unión por rolling, rellena las fechas que faltaban con el último valor conocido "roll=Inf"
X2_orig_bd=X2_orig_bd[tfhd, on = .(Date), roll = Inf] 
print(head(X2_orig_bd))
#buscar fecha de valuación en tfondeo
tf_act=X2_orig_bd[fecha==fval,]$V2/100
tf_int=X2_orig_bd[fecha<=fval & fecha>=(fval-plazocupon_bdm[1])]$V2/100

X1_orig=setDT(X1_orig)[, Date:= V1][order(-Date)] #Para alinear con valor presente y tasa de fondeo.

print(head(X1_orig[,1:4]))


  
  #CARGA DE DATOS PARA SWAP
  data1<-read.table(btasadesc_sw)
  n1<-nrow(data1)
  m1_orig_sw=ncol(data1)
  X1_orig_sw=data.table(mutate(data1[2:n1,1:m1_orig_sw],Date=as.Date(V1,format="%Y%m%d")))
  X1_orig_sw=X1_orig_sw%>%select(-V1)
  nodos1_sw=data.frame(data1[1,2:m1_orig_sw])
  
  data2<-read.table(btasacupvar_sw)
  n2<-nrow(data2)
  m2_orig_sw=ncol(data2)
  X2_orig_sw=data.table(mutate(data2[2:n1,1:m1_orig_sw],Date=as.Date(V1,format="%Y%m%d")))
  X2_orig_sw=X2_orig_sw%>%select(-V1)
  nodos2_sw=data.frame(data2[1,2:m2_orig_sw])
  
  
  #CARGA DE DATOS PARA OPCIONES DE TASA DE INTERÉS
  #carga de datos
  #carga de rho	
  data1<-read.table(btasadesc_oir) 
  n<-nrow(data1)
  m1_orig_oir=ncol(data1)
  x1_orig_oir=data.table(mutate(data1[2:n,1:m1_orig_oir],Date=as.Date(V1,format="%Y%m%d")))
  x1_orig_oir=x1_orig_oir%>%select(-V1)
  nodos1_oir=data.frame(data1[1,2:m1_orig_oir])
  
  #data1[1:3,]
  #carga de tasas spot
  data2<-read.table(btasaspot_oir)
  n<-nrow(data2)
  m2_orig_oir=ncol(data2)
  x2_orig_oir=data.table(mutate(data2[2:n,1:m2_orig_oir],Date=as.Date(V1,format="%Y%m%d")))
  x2_orig_oir=x2_orig_oir%>%select(-V1)
  nodos2_oir=data.frame(data2[1,2:m2_orig_oir])
  #carga de volatilidades de spot
  data3<-read.table(bvolspot_oir)
  n<-nrow(data3)
  m3_orig_oir=ncol(data3)
  x3_orig_oir=data.table(mutate(data3[2:n,1:m3_orig_oir],Date=as.Date(V1,format="%Y%m%d")))
  x3_orig_oir=x3_orig_oir%>%select(-V1)
  
  nodos3_oir=data.frame(data3[1,2:m3_orig_oir])




In [ ]:
%%R

#CARGA DE DATOS DE  FORWARDS DE TDC
#datas
#data<-read.table("tasa_tiie.txt")
data1<-read.table(bext)
data2<-read.table(bdom)

########minimos para parametrizar
n1=nrow(data1)
n2=nrow(data2)
m1_ftdc=ncol(data1)
m2_ftdc=ncol(data2)
n=min(n1,n2)-1
###NODOS###
nodos1_ftdc=data.frame(data1[1,2:m1_ftdc])
nodos2_ftdc=data.frame(data2[1,2:m2_ftdc])
####MATRICES DEL MISMO TAMAÑO MENOS DOLAR

x1_ftdc=as.data.table(mutate(data1[2:n,1:m1_ftdc],Date=as.Date(V1,format="%Y%m%d")))

x1_ftdc=x1_ftdc%>%select(-V1)
x2_ftdc=as.data.table(mutate(data2[2:n,1:m2_ftdc],Date=as.Date(V1,format="%Y%m%d")))
x2_ftdc=x2_ftdc%>%select(-V1)
print(x2_ftdc[1:20,])

###Para Dolar

if (yext==1)
{
#Cargar los símbolos de yahoo finance para FX
  start_date=fval-3660 #fecha inicial


  
#Creación del objeto para guardar los datos
  dataEnvFX_ftdc<-new.env()
  
#obtener los datos
  getSymbols.yahoo(SymbolsFX_ftdc,env=dataEnvFX_ftdc,from=start_date, to=(fval))
#limpiarlos, alinearnos y quedarnos con el precio de cierre 
  bt.prep(dataEnvFX_ftdc,align='remove.na',fill.gaps=T)
  
#muestra de datos
  head(dataEnvFX_ftdc$prices[,2])
  
#Nos quedamos con los precios
  X3_ftdc=data.table(Date=as.Date(index(dataEnvFX_ftdc$prices[,2])),coredata(dataEnvFX_ftdc$prices[,2]))
} else  
{ 
data3<-read.table(btsp)
print(head(data3))
n3<-nrow(data3)
m3<-ncol(data3)
X3=data.table(as.matrix(as.double(as.matrix(data3[2:(n+1),m3]))))
X3_find=as.data.table(mutate(data3[2:(n+1),1:m3],Date=as.Date(V1,format="%Y%m%d")))
}



# Parte extra

In [ ]:
class CrossCurrencySwap:
    
    def __init__(self):
        self.precio = None
        self.nocional_extranjera = None
        self.nocional_local = None
        self.n_cupones = None
        self.posicion = None
        self.convencion = None
        self.flotante_ext = None
        self.flotante_loc = None
        self.descuentos_ext = None
        self.descuentos_loc = None
        self.spot = None
        self.summary = None
        
    def compute(self, fval, n,M_l, M_e, Spot, flot_l,cal_flot_l, flot_e,cal_flot_e, desc_l, cal_desc_l,desc_e, cal_desc_e,plazos, tau,  payer = False, act_360 = True):
        """
        M: Nocional
        n: # de cupones
        fix_rate: Tasa fija de la pata larga del IRS
        flotante: Curva de tasas de mercado (la que encuentras en BANXICO) de la pata corta
        calen_flotante: Las fechas dadas por la curva de flotante
        descuentos: Curca de tasas de descuento de mercado para valuar el IRS
        calen_desc: Las fechas dadas por la curva de descuentos
        plazos : Lista de plazo del i -ésimo cupón (habiles)
        tau: Plazo en dias del i-ésimo cupón. (naturales)
        payer: Dummy; 1 si paga fija, 0 si paga flotante
        act_360: Dummy; tipo de convención 1 si act/360, 0 si act/365
        """

        spot = fval
        
        if act_360:
            conv = 360
        else:
            conv = 365
            
        aux = pd.DataFrame({"Cupon":list(range(1,n+1)), "Tau":tau})
        aux["Desde Spot"] = aux["Tau"].cumsum()
        aux["Fecha"] = aux["Desde Spot"].apply(lambda x: timedelta(x) + spot)
                
        df = pd.DataFrame({"Cupon":list(range(1,n+1)), "Plazo": plazos, "Tau": tau})
                
        df["Fechas"] =  aux["Fecha"]
        df["Flotante_Local"] = df["Fechas"].apply(InterpolacionLineal, args=(cal_flot_l, flot_l))
        df["Descuentos_Local"]=df["Fechas"].apply(InterpolacionLineal, args=(cal_desc_l, desc_l))
        df["Flotante_Ext"] = df["Fechas"].apply(InterpolacionLineal, args=(cal_flot_e, flot_e))
        df["Descuentos_Ext"]=df["Fechas"].apply(InterpolacionLineal, args=(cal_desc_e, desc_e))
        
        # Calculo de tasas forward
        df['Forward_Flotante_Local'] = (conv/(aux['Desde Spot']-aux['Desde Spot'].shift(1)))*(((1+(df['Flotante_Local']*aux['Desde Spot']/conv))/(1+(df['Flotante_Local'].shift(1)*aux['Desde Spot'].shift(1)/conv))) - 1)
        df['Forward_Flotante_Local'][0] = df['Flotante_Local'][0]
        df['Forward_Flotante_Ext'] = (conv/(aux['Desde Spot']-aux['Desde Spot'].shift(1)))*(((1+(df['Flotante_Ext']*aux['Desde Spot']/conv))/(1+(df['Flotante_Ext'].shift(1)*aux['Desde Spot'].shift(1)/conv))) - 1)
        df['Forward_Flotante_Ext'][0] = df['Flotante_Ext'][0]
        
        df["Sumando_Local"] = (M_l*((df["Forward_Flotante_Local"])*df["Plazo"]/conv)) / (1 + (df["Descuentos_Local"]*df["Tau"]/conv))
        df["Sumando_Ext"] = (M_e*Spot*((df["Forward_Flotante_Ext"])*df["Plazo"]/conv)) / (1 + (df["Descuentos_Ext"]*df["Tau"]/conv))
        df["Sumando"] = df["Sumando_Local"] - df["Sumando_Ext"]

        self.precio =  ((-1)**payer)*df["Sumando"].sum()
        self.nocional_extranjera = M_e
        self.nocional_local = M_l
        self.n_cupones = n
        self.posicion = "Payer" if payer else "Receiver"
        self.convencion = conv
        self.flotante_ext = df[["Fechas","Forward_Flotante_Ext"]]
        self.flotante_loc = df[["Fechas","Forward_Flotante_Local"]]
        self.descuentos_ext = df[["Fechas", "Descuentos_Ext"]]
        self.descuentos_loc = df[["Fechas", "Descuentos_Local"]]
        self.spot = Spot
        self.summary = df
        return
    
    # PENDIENTE
    def plots(self):
        Bool = True
        while Bool:
            print("\n")
            choice = input("Si deseas ver las tasas de descuento teclea 'Descuentos', si deseas\
                                ver las tasas flotantes teclea 'Tasas' o si deseas salir escribe '0':\n")
            if choice == 'Descuentos' or choice =='Tasas' or choice == '0':
                Bool = False
            else:
                print('\nTu elección no es válida')
        if choice == 'Descuentos':
            sns.set_style('darkgrid')
            sns.set_palette('tab10')
            plt.figure(figsize = (12,8))
            ax = sns.lineplot(x = self.summary['Fechas'], y = self.summary['Descuentos'])
            ax.set_title("Curva de Descuentos",fontsize = '25')
            plt.show()
            
            return
        elif choice == 'Tasas':
            sns.set_style('darkgrid')
            sns.set_palette('tab10')
            plt.figure(figsize = (12,8))
            ax = sns.lineplot(x = self.summary['Fechas'], y = self.summary['Forward_Flotante'], color = 'red')
            ax.set_title("Curva de Tasas Forward Flotantes",fontsize = '25')
            plt.show()
            
            return
        else:
            return

In [ ]:
class Riesgo_CCS:

  def __init__(self):
    self.nocionales = None
    self.vencimiento = None
    self.dias_bono = None
    self.VaR = None
    self.CVaR = None
    self.alpha = None
    self.spot = None
    self.summary = None
    self.ordenado = None
    self.alisado = None
    self.precios = None
    
  def compute(self, fval, spot,cupon_l, descuento_l,cupon_e,descuento_e, vencimiento,\
              dias_bono, posiciones, alpha, alisado = False):
    """
    a
    """
    calen_flotante_l = cupon_l.columns
    calen_flotante_l = list(calen_flotante_l)
    calen_flotante_l.remove('DATE')
    calen_desc_l = descuento_l.columns
    calen_desc_l = list(calen_desc_l)
    calen_desc_l.remove('DATE')


    calen_flotante_e = cupon_e.columns
    calen_flotante_e = list(calen_flotante_e)
    calen_flotante_e.remove('DATE')
    calen_desc_e = descuento_e.columns
    calen_desc_e = list(calen_desc_e)
    calen_desc_e.remove('DATE')


    n = int(vencimiento/dias_bono)
    if n == (vencimiento/dias_bono):
        plazos = [dias_bono]*n
        tau = plazos
    else:
        plazos = [vencimiento-n*dias_bono] + [dias_bono]*(n-1)
        tau = plazos

    # Aquí vamos
    n_cupl = cupon_l.shape[1] - 1
    n_descl = descuento_l.shape[1] - 1
    n_cupe = cupon_e.shape[1] - 1
    n_desce = descuento_e.shape[1] - 1

    # Spot
    cross_currency = yf.download(spot, end = fval)
    cross_currency = cross_currency[['Close']]
    cross_currency = cross_currency.iloc[::-1]
    cross_currency =cross_currency.reset_index()
    cross_currency = cross_currency.rename(columns = {'Date':'DATE','Close':spot})
    
    #  X

    X = cupon_l.merge(descuento_l,how = 'inner',on = 'DATE')
    X = X.merge(cupon_e,how = 'inner',on = 'DATE')
    X = X.merge(descuento_e,how = 'inner',on = 'DATE')
    X = X.merge(cross_currency, how = 'inner',on = 'DATE')
    X.set_index('DATE', inplace=True)

    X0 = X.head(1)
    DeltaX = (X.shift(1)/X)-1
    DeltaX.iloc[0,:] = 0
    X_s = X0.values*(1+DeltaX)



    Y_s = pd.DataFrame()
    irs_c= InterestRateSwap()
    cup_l = X_s.iloc[:,:n_cupl]
    desc_l = X_s.iloc[:,n_cupl:n_cupl+n_descl]
    cup_e = X_s.iloc[:,n_cupl+n_descl:n_cupl+n_descl+n_cupe]
    desc_e = X_s.iloc[:,n_cupl+n_descl+n_cupe:n_cupl+n_descl+n_cupe+n_desce]

    #(self, fval, n,M_l, M_e, Spot, flot_l,cal_flot_l, flot_e,cal_flot_e, descu_l, cal_desc_l, 
    #            desc_e, cal_desc_e,plazos, tau,  payer = False, act_360 = True)

    # Valoraciones
    aux = []
    for i in range(len(cup_l)):
      fval_aux = cup_l.index[i]
      flot_l =  cup_l.iloc[i,:].values/100
      flot_l = list(flot_l)
      descu_l = desc_l.iloc[i,:].values/100
      descu_l = list(descu_l)
      cal_flot_l_aux = [fval_aux + timedelta(int(i)) for i in calen_flotante_l]
      cal_desc_l_aux = [fval_aux + timedelta(int(i)) for i in calen_desc_l]

      flot_e =  cup_e.iloc[i,:].values/100
      flot_e = list(flot_e)
      descu_e = desc_e.iloc[i,:].values/100
      descu_e = list(descu_e)
      cal_flot_e_aux = [fval_aux + timedelta(int(i)) for i in calen_flotante_e]
      cal_desc_e_aux = [fval_aux + timedelta(int(i)) for i in calen_desc_e]

      ccs = CrossCurrencySwap()
      ccs.compute(fval_aux, n ,posiciones[0],posiciones[1], X[spot][i], flot_l,cal_flot_l_aux, flot_e,cal_flot_e_aux, descu_l, cal_desc_l_aux,\
                  descu_e, cal_desc_e_aux,plazos, tau)

      aux.append(ccs.precio)

    Y_s['CCS'] = aux

    #print(Y_s)
 
    Y0 = Y_s.head(1)
    Y0 = Y0.values[0]
    #print(Y0)
    PL = Y_s-Y0
    PL['Total']=PL.sum(axis=1)
    #print(PL)
    VaR_C=[]

    if alisado:
      c = 1
      PL.dropna(inplace = True)
      PL = PL.iloc[::-1]
      PL["Esc_SA"] = ExponentialWeighted(0.05,PL.shape[0])
      col_aux = list(PL.columns)
      col_aux.remove("Esc_SA")
      vector_VaR = [VaR_CA(PL.iloc[:,i],PL.Esc_SA,alpha) for i in range(PL.shape[1]-1)]
      VaR = pd.Series(vector_VaR,index = col_aux)
    else:
      c = 0
      VaR=PL.quantile(alpha, numeric_only=False)


    for i in range(0,PL.shape[1]-c):
      VC=CVaR(PL.iloc[:,i],VaR[i])
      VaR_C.append(VC)
    VaR=PL.quantile(alpha, numeric_only=False)
    
    self.summary = pd.DataFrame({"VaR":VaR})
    self.summary['CVaR'] = VaR_C

    self.nocionales = {'Local':posiciones[0],'Extranjero':posiciones[1]}
    self.vencimiento = vencimiento
    self.dias_bono = dias_bono
    self.spot = spot
    self.VaR = self.summary[['VaR']]
    self.CVaR = self.summary[['CVaR']]
    self.alpha = alpha
    self.alisado = alisado
    self.precios = Y0

    return